# Système Multi-Agents pour la Gestion Intelligente de Bâtiment

Ce notebook implémente l'architecture décrite dans le rapport de stage, basée sur un système multi-agents sémantique pour le pilotage énergétique et de confort d'une pièce.

## 1. Initialisation et Configuration

Cette première cellule charge toutes les bibliothèques nécessaires, configure les variables globales (chemins, namespaces RDF) et initialise le **Tableau Blanc Sémantique**, qui est notre base de connaissances partagée.

In [1]:
# ==============================================================================
# 1. IMPORTATIONS ET CONFIGURATION INITIALE
# ==============================================================================
# Cette section importe toutes les bibliothèques nécessaires au projet et
# définit les constantes globales comme les chemins de fichiers et les
# espaces de noms pour la manipulation des données sémantiques.

import datetime
import os
import time
from rdflib import Graph, Literal, URIRef, Namespace
from rdflib.namespace import RDF, RDFS, XSD, OWL
import json
import asyncio
import re
from dateutil.parser import parse
import requests
import openai 
from dotenv import load_dotenv


# --- Configuration des chemins et URIs ---

# URI de base pour tous les concepts de notre ontologie.
ONTOLOGY_URI = "http://monbatiment.com/ontologie/SMA4SB#"
# L'ontologie de base avec toutes les définitions
ONTOLOGY_FILE_NAME = "SMA4SB.ttl" 
# Fichier de données où l'état actuel du système (le "tableau blanc") est sauvegardé.
DYNAMIC_WHITEBOARD_FILE = "tableau_blanc_dynamique.data.ttl"

# --- Définition des espaces de noms (Namespaces) pour RDFLib ---
# Raccourcis utilisés dans les requêtes SPARQL pour rendre le code plus lisible.
EX = Namespace(ONTOLOGY_URI)

GLOBAL_NAMESPACES = {
    "ex": EX,
    "rdf": RDF,
    "rdfs": RDFS,
    "xsd": XSD,
    "owl": OWL
}

# ==============================================================================
# 2. INITIALISATION DU TABLEAU BLANC SÉMANTIQUE
# ==============================================================================
# Le "Tableau Blanc" est la mémoire partagée de tous les agents.
# C'est un graphe RDF (stocké dans la variable `g_tableau_blanc`) qui contient
# l'état actuel du bâtiment, les intentions des agents, etc.

print(f"🔄 Tentative d'initialisation du Tableau Blanc Sémantique...")

# Creation d'une instance du graphe RDF qui servira de tableau blanc.
g_tableau_blanc = Graph()

try:
    # Chargement d'un tableau blanc existant s'il n'est pas vide.
    # Cela permet de reprendre une simulation là où elle s'était arrêtée.
    if os.path.exists(DYNAMIC_WHITEBOARD_FILE) and os.path.getsize(DYNAMIC_WHITEBOARD_FILE) > 0:
        g_tableau_blanc.parse(DYNAMIC_WHITEBOARD_FILE, format="turtle")
        print(f"  [Système] Tableau Blanc dynamique chargé avec succès depuis '{DYNAMIC_WHITEBOARD_FILE}'.")
    else:
        # Si aucun tableau blanc n'existe, on en crée un nouveau
        # en chargeant la structure de base depuis le fichier d'ontologie.
        print(f"  [Système] Le Tableau Blanc dynamique est vide ou n'existe pas ('{DYNAMIC_WHITEBOARD_FILE}').")
        print(f"  [Système] Chargement de l'ontologie de base depuis '{ONTOLOGY_FILE_NAME}' pour l'initialisation...")
        g_tableau_blanc.parse(ONTOLOGY_FILE_NAME, format="turtle")
        print(f"  [Système] Ontologie de base chargée dans le Tableau Blanc.")
        # On sauvegarde ce nouvel état de base.
        g_tableau_blanc.serialize(destination=DYNAMIC_WHITEBOARD_FILE, format="turtle", encoding="utf-8")
        print(f"  [Système] Ontologie de base sauvegardée comme Tableau Blanc dynamique initial dans '{DYNAMIC_WHITEBOARD_FILE}'.")
# Gestion des erreurs si le fichier d'ontologie est introuvable ou corrompu.
except Exception as e:
    print(f"  [ERREUR] Échec du chargement ou de l'initialisation du Tableau Blanc Sémantique : {e}")
    print(f"  [Système] Le graphe pourrait être vide ou incomplet. Le système tentera de continuer.")


🔄 Tentative d'initialisation du Tableau Blanc Sémantique...
  [Système] Le Tableau Blanc dynamique est vide ou n'existe pas ('tableau_blanc_dynamique.data.ttl').
  [Système] Chargement de l'ontologie de base depuis 'SMA4SB.ttl' pour l'initialisation...
  [Système] Ontologie de base chargée dans le Tableau Blanc.
  [Système] Ontologie de base sauvegardée comme Tableau Blanc dynamique initial dans 'tableau_blanc_dynamique.data.ttl'.


## 2. Fonctions Utilitaires

Ensemble de fonctions pour interagir avec le **Tableau Blanc Sémantique** (le graphe RDF). Elles permettent de lire des données, d'ajouter des concepts (intentions, objections) et de manipuler le graphe. C'est la "boîte à outils" de nos agents.

In [2]:
# ==============================================================================
# 3. FONCTIONS UTILITAIRES POUR L'INTERACTION AVEC LE GRAPHE RDF
# ==============================================================================
# Cet ensemble de fonctions constitue l'API (Interface de Programmation)
# que les agents utilisent pour "parler" au Tableau Blanc Sémantique.
# Elles cachent la complexité des requêtes SPARQL et assurent que les
# données sont lues et écrites de manière structurée et cohérente.

def get_physical_property(graph: Graph, zone_uri: URIRef, property_uri: URIRef):
    """Récupère une propriété physique (comme le volume, la capacité thermique) pour une zone donnée."""
    value = graph.value(subject=zone_uri, predicate=property_uri)
    if value:
        return float(value)
    else:
        print(f"  [UTIL] AVERTISSEMENT: Propriété {property_uri.split('#')[-1]} non trouvée pour {zone_uri.split('#')[-1]}.")
        return None


def get_current_sensor_value(graph: Graph, sensor_uri: URIRef, property_name: str):
    """Récupère la dernière valeur d'un capteur spécifique."""
    query = f"""
    SELECT ?value WHERE {{
        <{sensor_uri}> ex:{property_name} ?observation .
        ?observation ex:aPourValeur ?value ;
                     ex:timestamp ?timestamp .
    }} ORDER BY DESC(?timestamp) LIMIT 1
    """
    results = graph.query(query, initNs=GLOBAL_NAMESPACES)
    for row in results:
        return float(row.value) if isinstance(row.value, Literal) else float(row.value)
    print(f"  [UTIL] AVERTISSEMENT: Aucune valeur trouvée pour le capteur {sensor_uri.split('#')[-1]} avec la propriété {property_name}.")
    return None

def get_occupant_preference(graph: Graph, occupant_uri: URIRef, preference_property: str):
    """Récupère la préférence d'un occupant."""
    query = f"""
    SELECT ?pref WHERE {{
        <{occupant_uri}> ex:{preference_property} ?pref .
    }}
    """
    results = graph.query(query, initNs=GLOBAL_NAMESPACES)
    for row in results:
        return float(row.pref) if isinstance(row.pref, Literal) else float(row.pref)
    print(f"  [UTIL] AVERTISSEMENT: Aucune préférence trouvée pour l'occupant {occupant_uri.split('#')[-1]} avec la propriété {preference_property}.")
    return None

def get_equipment_state(graph: Graph, equipment_uri: URIRef):
    """Récupère l'état actuel d'un équipement."""
    query = f"""
    SELECT ?state_value WHERE {{
        <{equipment_uri}> ex:aPourEtat ?state_value_node .
        BIND(IF(isIRI(?state_value_node), STRAFTER(STR(?state_value_node), STR(ex:)), STR(?state_value_node)) AS ?state_value)
    }}
    """
    results = graph.query(query, initNs=GLOBAL_NAMESPACES)
    for row in results:
        return str(row.state_value)
    print(f"  [UTIL] AVERTISSEMENT: Aucun état trouvé pour l'équipement {equipment_uri.split('#')[-1]}.")
    return "Inconnu"

def add_intention(graph: Graph, agent_uri: URIRef, bureau_uri: URIRef, description: str, generic_type: str, concerne_equipement: URIRef = None):
    """
    Ajoute une nouvelle intention au tableau blanc, en mappant les types génériques
    aux classes d'intentions spécifiques de l'ontologie.
    """
    intention_uri = EX[f"Intention_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"]

    # Ce bloc de "if/elif" agit comme un traducteur entre les décisions des agents
    # et le vocabulaire formel de l'ontologie.
    
    # Type par défaut si aucun mappage ne correspond
    ontological_type_uri = EX.Intention 
    
    if generic_type == "AugmenterTemperature" and concerne_equipement == EX.RadiateurBureau1:
        ontological_type_uri = EX.ActiverChauffage
    elif generic_type == "DiminuerTemperature" and concerne_equipement == EX.FenetreBureau1:
        # Si c'est pour rafraîchir (ouvrir) ou pour économiser (fermer)
        if "Ouvrir la fenêtre" in description:
            ontological_type_uri = EX.OuvrirFenetre
        elif "Fermer la fenêtre" in description:
            ontological_type_uri = EX.FermerFenetre
    elif generic_type == "AmeliorerQualiteAir" and concerne_equipement == EX.FenetreBureau1:
        ontological_type_uri = EX.OuvrirFenetre
    elif generic_type == "AmeliorerQualiteAir" and concerne_equipement == EX.VentilationBureau1:
        ontological_type_uri = EX.ActiverVentilation
    elif generic_type == "AugmenterLuminosite" and concerne_equipement == EX.LampeBureau1:
        ontological_type_uri = EX.AllumerEclairage
    elif generic_type == "DiminuerLuminosite" and concerne_equipement == EX.LampeBureau1:
        ontological_type_uri = EX.EteindreEclairage
    elif generic_type == "DiminuerLuminosite" and concerne_equipement == EX.FenetreBureau1:
        ontological_type_uri = EX.FermerFenetre
    elif generic_type == "DiminuerTemperature" and concerne_equipement == EX.ClimatiseurBureau1:
        ontological_type_uri = EX.ActiverClimatiseur # Classe à ajouter à l'ontologie si besoin

    # On ajoute toutes les informations (les "triplets") qui décrivent cette nouvelle intention.
    graph.add((intention_uri, RDF.type, ontological_type_uri))
    graph.add((intention_uri, EX.description, Literal(description, datatype=XSD.string)))
    graph.add((intention_uri, EX.creeLe, Literal(datetime.datetime.now(), datatype=XSD.dateTime)))
    graph.add((intention_uri, EX.viseZone, bureau_uri))
    graph.add((intention_uri, EX.genereParAgent, agent_uri))
    if concerne_equipement:
        graph.add((intention_uri, EX.concerneEquipement, concerne_equipement))

    # Toute nouvelle intention commence avec le statut "Proposée".
    graph.add((intention_uri, EX.aPourStatut, EX.Statut_Proposee))

    # Log pour le suivi
    type_intention_name = ontological_type_uri.split('#')[-1]
    concerne_equipement_name = concerne_equipement.split('#')[-1] if concerne_equipement else 'N/A'
    print(f"  [{agent_uri.split('#')[-1]}] Intention ajoutée : '{description}' (Type ontologique: {type_intention_name}, Équipement: {concerne_equipement_name})")
    return intention_uri

def add_conflict(graph: Graph, agent_uri: URIRef, bureau_uri: URIRef, description: str, gravite: str, 
                 intention1_uri: URIRef, intention2_uri: URIRef = None): 
    
    #Ajoute un conflit en le liant directement aux intentions.
    conflict_uri = EX[f"Conflit_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"]
    graph.add((conflict_uri, RDF.type, EX.Conflit))
    graph.add((conflict_uri, EX.description, Literal(description, datatype=XSD.string)))
    graph.add((conflict_uri, EX.gravite, Literal(gravite, datatype=XSD.integer))) 
    graph.add((conflict_uri, EX.creeLe, Literal(datetime.datetime.now(), datatype=XSD.dateTime)))
    graph.add((conflict_uri, EX.viseZone, bureau_uri))
    graph.add((conflict_uri, EX.genereParAgent, agent_uri))
    graph.add((conflict_uri, EX.concerneIntention, intention1_uri))
    if intention2_uri:
        graph.add((conflict_uri, EX.concerneIntention, intention2_uri))
        
    print(f"  [{agent_uri.split('#')[-1]}] Conflit détecté et ajouté : '{description}'")
    return conflict_uri

def add_action_log(graph: Graph, agent_uri: URIRef, bureau_uri: URIRef, action_description: str, equipment_uri: URIRef = None):
    
    #Ajoute une entrée de log pour une action exécutée au tableau blanc (persistant).
    log_uri = EX[f"ActionLog_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"]
    graph.add((log_uri, RDF.type, EX.ActionExecutee)) 
    graph.add((log_uri, EX.description, Literal(action_description, datatype=XSD.string)))
    graph.add((log_uri, EX.creeLe, Literal(datetime.datetime.now(), datatype=XSD.dateTime)))
    graph.add((log_uri, EX.viseZone, bureau_uri))
    graph.add((log_uri, EX.genereParAgent, agent_uri)) 
    if equipment_uri:
        graph.add((log_uri, EX.concerneEquipement, equipment_uri))
    
    equipment_name = equipment_uri.split('#')[-1] if equipment_uri else 'N/A'
    print(f"  [{agent_uri.split('#')[-1]}] LOG ACTION : '{action_description}' sur '{equipment_name}'.")
    return log_uri

def add_resolution_log(graph: Graph, agent_uri: URIRef, bureau_uri: URIRef, resolution_description: str, 
                       conflict_uri: URIRef, 
                       validated_intention_uri: URIRef,  
                       discarded_intentions_uris: list = None): 
    
    #Ajoute un log de résolution en liant l'intention validée et les intentions écartées.
    log_uri = EX[f"ConflitResolu_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"]
    
    graph.add((log_uri, RDF.type, EX.ConflitResolu))
    
    graph.add((log_uri, EX.description, Literal(resolution_description, datatype=XSD.string)))
    
    graph.add((log_uri, EX.creeLe, Literal(datetime.datetime.now(), datatype=XSD.dateTime)))
    
    graph.add((log_uri, EX.viseZone, bureau_uri))
    graph.add((log_uri, EX.genereParAgent, agent_uri))
    graph.add((log_uri, EX.concerneConflit, conflict_uri))
    
    if validated_intention_uri:
        graph.add((log_uri, EX.intentionValidee, validated_intention_uri))
    
    if discarded_intentions_uris:
        for intent_uri in discarded_intentions_uris:
            graph.add((log_uri, EX.aResoluIntention, intent_uri))
            
    validated_intent_name = validated_intention_uri.split('#')[-1] if validated_intention_uri else "Aucune"
    print(f"  [{agent_uri.split('#')[-1]}] LOG RESOLUTION: '{resolution_description}'. Intention validée: '{validated_intent_name}'.")
    return log_uri

def add_need_to_piece(graph: Graph, agent_uri: URIRef, bureau_uri: URIRef, need_ontological_type: URIRef, description: str):
    """
    [VERSION CORRIGÉE] Crée une instance de Besoin, la lie à la pièce via :aPourBesoin,
    et enregistre ses métadonnées.
    """
    # Creation d'une instance du besoin spécifique (ex: une instance de :BesoinChauffage)
    need_uri = EX[f"{need_ontological_type.split('#')[-1]}_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"]
    
    # On lie la pièce à son nouveau besoin
    graph.add((bureau_uri, EX.aPourBesoin, need_uri))

    # On décrit l'instance du besoin
    graph.add((need_uri, RDF.type, need_ontological_type))
    graph.add((need_uri, EX.description, Literal(description, datatype=XSD.string)))
    graph.add((need_uri, EX.creeLe, Literal(datetime.datetime.now(), datatype=XSD.dateTime)))
    graph.add((need_uri, EX.viseZone, bureau_uri))
    graph.add((need_uri, EX.genereParAgent, agent_uri))
    
    print(f"  [{agent_uri.split('#')[-1]}] BESOIN AJOUTÉ : Pièce='{bureau_uri.split('#')[-1]}', Type='{need_ontological_type.split('#')[-1]}'.")
    return need_uri

def clear_previous_intentions_and_conflicts_by_agent(graph: Graph, agent_uri: URIRef, bureau_uri: URIRef):
    # Supprime les intentions et conflits.
    
    # Nettoyage des intentions générées par cet agent
    q_clear_intentions = f"""
    DELETE {{ ?intention ?p ?o }}
    WHERE {{
        # CORRECTION 2: On cherche toute instance dont le type est une sous-classe de :Intention
        ?intention rdf:type ?typeIntention .
        ?typeIntention rdfs:subClassOf* ex:Intention .

        ?intention ex:genereParAgent <{agent_uri}> .
        
        # CORRECTION 1: Utilisation de la bonne propriété de zone
        ?intention ex:viseZone <{bureau_uri}> .
        
        ?intention ?p ?o .
    }}
    """
    graph.update(q_clear_intentions, initNs=GLOBAL_NAMESPACES)
    print(f"  [{agent_uri.split('#')[-1]}] Nettoyage des intentions précédentes générées par cet agent.")

    q_clear_conflicts = f"""
    DELETE {{ ?conflict ?p ?o }}
    WHERE {{
        ?conflict rdf:type ex:Conflit ;
                  ex:genereParAgent <{agent_uri}> ;
                  
                  # CORRECTION 1: Utilisation de la bonne propriété de zone
                  ex:viseZone <{bureau_uri}> .
                  
        ?conflict ?p ?o .
    }}
    """
    graph.update(q_clear_conflicts, initNs=GLOBAL_NAMESPACES)
    print(f"  [{agent_uri.split('#')[-1]}] Nettoyage des conflits précédents générés par cet agent.")
    
def creer_human_prompt_superviseur(intentions: list, current_states: dict, occupant_prefs: dict):
    """
    Crée le prompt pour le superviseur LLM en incluant
    le contexte de la pièce et en demandant une réponse structurée en JSON.
    """
    
    # 1. Contexte de la situation
    contexte = f"""
Contexte actuel du Bureau 1:
- Température: {current_states.get('temp_actuelle', 'N/A')}°C (Préférence: {occupant_prefs.get('temp', 'N/A')}°C)
- CO2: {current_states.get('co2_actuel', 'N/A')} ppm (Préférence: {occupant_prefs.get('co2', 'N/A')} ppm)
- Luminosité: {current_states.get('lumi_actuelle', 'N/A')} lux (Préférence: {occupant_prefs.get('lumi', 'N/A')} lux)
"""

    # 2. Liste des options d'action
    dossier_intentions = ""
    for i, intent in enumerate(intentions):
        dossier_intentions += f"- OPTION_{i+1}: {intent['description']} (Objectif: {intent['type']})\n"

    # 3. Instructions claires et format de sortie JSON
    return f"""
Vous êtes le superviseur expert d'un système de gestion de bâtiment intelligent.
Votre rôle est de valider le plan d'action final en vous basant sur le contexte et les options proposées.

{contexte}
Voici le plan d'action final proposé :
{dossier_intentions}
Analysez ce plan. Est-il parfaitement logique et efficace ? Y a-t-il des problèmes subtils ou des redondances ?

Répondez OBLIGATOIREMENT en utilisant le format JSON suivant. Ne donnez aucune autre explication en dehors du JSON.

{{
  "plan_valide": boolean,
  "meilleure_option": "OPTION_X",
  "raisonnement": "Votre analyse concise ici."
}}
"""
    
def get_recent_events_for_prompt(graph: Graph, bureau_uri: URIRef, limit: int = 5):
    """
    Récupère les événements récents (tous les Besoins et Actions) 
    pour enrichir le contexte d'un prompt LLM.
    """
    query = f"""
    SELECT ?timestamp ?description WHERE {{
        {{
            # Cherche tous les types de Besoins
            ?event rdf:type ?type .
            ?type rdfs:subClassOf* ex:Besoin .
        }}
        UNION
        {{
            # Cherche les actions exécutées
            ?event rdf:type ex:ActionExecutee .
        }}
        UNION
        {{
            # AJOUTÉ : Cherche les résolutions de conflit
            ?event rdf:type ex:ConflitResolu .
        }}
        
        # Conditions communes à tous les événements
        ?event ex:viseZone <{bureau_uri}> .
        ?event ex:creeLe ?timestamp .
        ?event ex:description ?description .
    }}
    ORDER BY DESC(?timestamp)
    LIMIT {limit}
    """
    try:
        results = graph.query(query, initNs=GLOBAL_NAMESPACES)
        if not results:
            return "Aucun événement récent pertinent trouvé."
        
        history_lines = []
        for row in results:
            event_time = row.timestamp.toPython()
            # Utilisation d'un datetime conscient du fuseau horaire pour la soustraction
            now_aware = datetime.datetime.now(event_time.tzinfo)
            time_ago = now_aware - event_time
            
            if time_ago.total_seconds() < 60:
                time_str = f"Il y a {int(time_ago.total_seconds())} secondes"
            elif time_ago.total_seconds() < 3600:
                time_str = f"Il y a {int(time_ago.total_seconds() / 60)} minutes"
            else:
                time_str = "Il y a plus d'une heure"
            
            history_lines.append(f"- {time_str}: '{row.description}'")
        
        print(f"  [Mémoire] {len(history_lines)} événement(s) récent(s) récupéré(s) pour le prompt.")
        return "\\n".join(history_lines)
    except Exception as e:
        print(f"  [ERREUR Mémoire] Impossible de récupérer l'historique : {e}")
        return "Erreur lors de la récupération de l'historique."
        

def add_objection(graph: Graph, agent_uri: URIRef, bureau_uri: URIRef,
                  original_intention_uri: URIRef, objection_reason: str,
                  contre_proposition_desc: str, contre_proposition_type: str, contre_proposition_equip: URIRef):
    """
    Ajoute une objection structurée avec horodatage et lien vers la zone.
    """
    objection_uri = EX[f"Objection_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"]
    graph.add((objection_uri, RDF.type, EX.Objection))
    graph.add((objection_uri, RDFS.comment, Literal(objection_reason)))
    graph.add((objection_uri, EX.genereParAgent, agent_uri))
    graph.add((objection_uri, EX.creeLe, Literal(datetime.datetime.now(), datatype=XSD.dateTime)))
    graph.add((objection_uri, EX.viseZone, bureau_uri))
    
    # Lier l'objection à l'intention originale
    graph.add((objection_uri, EX.estUneObjectionA, original_intention_uri))

    # Créer la contre-proposition (qui est une Intention de type ContreProposition)
    cp_uri = add_intention(graph, agent_uri, bureau_uri, contre_proposition_desc, contre_proposition_type, contre_proposition_equip)
    graph.add((cp_uri, RDF.type, EX.ContreProposition))

    # Lier l'objection à sa contre-proposition
    graph.add((objection_uri, EX.proposeAlternative, cp_uri))

    print(f"  [{agent_uri.split('#')[-1]}] OBJECTION soulevée contre '{original_intention_uri.split('#')[-1]}'. Contre-proposition: '{contre_proposition_desc}'")
    return objection_uri


INTENTION_TYPE_MAP = {
    # Température
    EX.ActiverChauffage: "AugmenterTemperature",
    EX.DesactiverChauffage: "DiminuerTemperature",
    EX.FermerFenetre: "DiminuerTemperature",

    # Qualité de l'air (CO2)
    EX.ActiverVentilation: "AmeliorerQualiteAir",
    EX.DesactiverVentilation: "StopperAmeliorationAir",

    # Luminosité
    EX.AllumerEclairage: "AugmenterLuminosite",
    EX.EteindreEclairage: "DiminuerLuminosite",
    
}


def get_all_current_intentions(graph: Graph, bureau_uri: URIRef):
    """
    Récupère toutes les intentions en utilisant un mappage
    plus propre et plus facile à maintenir.
    """
    query = f"""
    SELECT ?intention ?type ?desc ?equip ?agent WHERE {{
        ?intention rdf:type ?type .
        ?type rdfs:subClassOf* ex:Intention .
        ?intention ex:viseZone <{bureau_uri}> .
        ?intention ex:description ?desc .
        ?intention ex:genereParAgent ?agent .
        OPTIONAL {{ ?intention ex:concerneEquipement ?equip . }}
        FILTER NOT EXISTS {{ ?intention rdf:type ex:ContreProposition . }}
    }}
    """
    results = graph.query(query, initNs=GLOBAL_NAMESPACES)
    intentions = []
    for row in results:
        generic_type = "Unknown"
        
        if row.type == EX.OuvrirFenetre:
            if "rafraîchir" in str(row.desc).lower():
                generic_type = "DiminuerTemperature"
            else:
                generic_type = "AmeliorerQualiteAir"
        else:
            generic_type = INTENTION_TYPE_MAP.get(row.type, "Unknown")

        equipment_name = str(row.equip).split('#')[-1] if row.equip else "None"
        intentions.append({
            "uri": row.intention,
            "type": generic_type,
            "description": str(row.desc),
            "equipment": equipment_name,
            "agent": str(row.agent).split('#')[-1]
        })
    return intentions

def get_all_objections(graph: Graph, bureau_uri: URIRef):
    """
    Récupère toutes les objections non résolues sur le Tableau Blanc, avec l'intention originale
    et la contre-proposition associées.
    """
    query = """
    SELECT ?objection ?reason ?original_intention ?contre_proposition WHERE {
        ?objection rdf:type ex:Objection ;
                   ex:estUneObjectionA ?original_intention ;
                   ex:proposeAlternative ?contre_proposition .
        
        # On s'assure que l'intention originale concerne bien notre bureau
        ?original_intention ex:viseZone ?zone .
        FILTER(?zone = ?bureau_uri)

        OPTIONAL { ?objection rdfs:comment ?reason . }
    }
    """
    results = graph.query(query, initNs={"ex": EX, "rdf": RDF, "rdfs": RDFS}, 
                          initBindings={'bureau_uri': bureau_uri})
    
    objections = []
    for row in results:
        objections.append({
            "uri": row.objection,
            "reason": str(row.reason) if row.reason else "N/A",
            "original_intention_uri": row.original_intention,
            "contre_proposition_uri": row.contre_proposition
        })
    return objections

## 3. Définition des Composants Fondamentaux

Ici, on définit les "services" intelligents que les agents pourront utiliser :
- **LLMManager** : Pour la communication sécurisée et résiliente avec les modèles de langage (OpenAI, Groq, etc.).


In [3]:
# ==============================================================================
# 4. COMPOSANTS FONDAMENTAUX
# ==============================================================================
# Cette section définit les "services" intelligents que les agents pourront
# utiliser pour augmenter leurs capacités de décision.

class LLMManager:
    """
    Gère la communication avec les Grands Modèles de Langage (LLM).
    
    Cette classe a plusieurs rôles clés :
    - Abstraction : Les agents n'ont pas besoin de savoir quel service LLM est
      utilisé (OpenAI, Groq, etc.). Ils utilisent une seule fonction simple.
    - Résilience : Implémente un mécanisme de "fallback". Si le service
      principal (primaire) échoue, il bascule automatiquement sur un service de secours.
    - Sécurité : Centralise la gestion des clés API.
    - Fiabilité : Intègre un parseur pour extraire proprement les réponses JSON
      attendues, même si le LLM ajoute du texte autour.
    """
    def __init__(self, openrouter_config: dict, groq_config: dict):
        """
        Initialise le manager avec deux configurations de services LLM.
        
        Args:
            openrouter_config: Dictionnaire contenant la clé API, l'URL de base et le nom du modèle pour le service principal.
            groq_config: Dictionnaire de configuration pour le service de secours.
        """

        # Configuration du client pour le service LLM principal
        self.primary_client = openai.AsyncOpenAI(
            base_url=openrouter_config["base_url"],
            api_key=openrouter_config["api_key"],
        )
        self.primary_model = openrouter_config["model_name"]

        # Configuration du client pour le service LLM de secours (fallback)
        self.fallback_client = openai.AsyncOpenAI(
            base_url=groq_config["base_url"],
            api_key=groq_config["api_key"],
        )
        self.fallback_model = groq_config["model_name"]
        print("  [Système] LLMManager initialisé avec OpenRouter, Groq et parsing JSON.")

    async def ainvoke(self, prompt_data: dict, temperature: float = 0, json_output: bool = False):
        """
        Fonction principale pour envoyer une requête à un LLM.
        
        Args:
            prompt_data: Dictionnaire contenant le prompt système et le prompt utilisateur.
            temperature: Contrôle le degré de créativité du LLM (0 = déterministe).
            json_output: Si True, la fonction essaiera de parser la réponse en JSON.
            
        Returns:
            La réponse du LLM (str ou dict) ou None en cas d'échec complet.
        """
        
        messages = [
            {"role": "system", "content": prompt_data["system_prompt_content"]},
            {"role": "user", "content": prompt_data["human_prompt_content"]},
        ]
        
        raw_content = None
        try:
            # On tente d'abord d'appeler le service principal.
            print(f"  [LLM] Tentative d'appel via OpenRouter (Sortie attendue: {'JSON' if json_output else 'Texte'})...")
            response = await self.primary_client.chat.completions.create(
                model=self.primary_model, messages=messages, temperature=temperature,
            )
            raw_content = response.choices[0].message.content
        except openai.APIStatusError as e:
            if e.status_code in [429,402]:
                print("  [LLM AVERTISSEMENT] Limite de débit sur OpenRouter. Basculement sur Groq...")
                try:
                     # ...on bascule automatiquement sur le service de secours.
                    response = await self.fallback_client.chat.completions.create(
                        model=self.fallback_model, messages=messages, temperature=temperature,
                    )
                    raw_content = response.choices[0].message.content
                except Exception as fallback_e:
                    print(f"  [LLM ERREUR] L'appel de secours via Groq a aussi échoué: {fallback_e}")
                    return None # Retourne None en cas d'échec complet
            else:
                print(f"  [LLM ERREUR] Erreur d'API non liée à la limite de débit: {e}")
                return None
        except Exception as e:
            print(f"  [LLM ERREUR] Erreur inattendue lors de l'appel LLM: {e}")
            return None

        if not raw_content:
            return None
            
        # Si l'on n'attend pas de JSON, on retourne le texte brut.
        if not json_output:
            return raw_content # Retourne le texte brut si le JSON n'est pas demandé

       # --- Logique de parsing pour extraire un JSON propre de la réponse ---
        try:
            # Regex pour trouver un bloc de code JSON, même s'il y a du texte avant ou après.
            json_match = re.search(r"```json\n({.*?})\n```", raw_content, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
            else:
                # Si le LLM n'a pas utilisé de bloc de code, on suppose que toute la réponse est du JSON. 
                json_str = raw_content
            
            return json.loads(json_str)
        except json.JSONDecodeError:
            # Si la réponse n'est pas un JSON valide, on signale l'erreur sans planter.
            print(f"  [LLM ERREUR PARSING] Impossible de décoder la réponse JSON: {raw_content}")
            return None 

## 4. Définition des Agents

C'est le cœur du système. Chaque classe représente un agent avec un rôle et des compétences spécifiques, conformément à la méthodologie Gaia. Ils interagissent indirectement via le Tableau Blanc.

In [4]:
# ==============================================================================
# 5. DÉFINITION DES AGENTS
# ==============================================================================
# C'est le cœur du système. Chaque classe ci-dessous représente un agent
# logiciel autonome avec un rôle et des compétences spécifiques. Ils ne
# communiquent jamais directement entre eux, mais uniquement en lisant et
# écrivant des informations sur le Tableau Blanc Sémantique (le graphe RDF).

class AgentPerception:
    """
    Rôle : Les "yeux et les oreilles" du système.
    - Simule les capteurs du monde physique (température, occupation, etc.).
    - Publie ces données sur le Tableau Blanc pour que les autres agents puissent les lire.
    - Simule également l'état initial des actionneurs (radiateur, fenêtre, etc.).
    """
    def __init__(self, graph: Graph, bureau_uri: URIRef, agent_uri: URIRef,
                 capteur_temp_uri: URIRef, capteur_co2_uri: URIRef, capteur_lumi_uri: URIRef,
                 capteur_humi_uri: URIRef, capteur_occup_uri: URIRef, 
                 capteur_temp_ext_uri: URIRef, 
                 capteur_lumi_ext_uri: URIRef,
                 radiateur_uri: URIRef, fenetre_uri: URIRef, lampe_uri: URIRef,
                 ventilation_uri: URIRef, volet_uri: URIRef):
        self.graph = graph
        self.bureau_uri = bureau_uri
        self.agent_uri = agent_uri
        self.capteur_temp_uri = capteur_temp_uri
        self.capteur_co2_uri = capteur_co2_uri
        self.capteur_lumi_uri = capteur_lumi_uri
        self.capteur_humi_uri = capteur_humi_uri     
        self.capteur_occup_uri = capteur_occup_uri   
        self.capteur_temp_ext_uri = capteur_temp_ext_uri
        self.capteur_lumi_ext_uri = capteur_lumi_ext_uri
        self.radiateur_uri = radiateur_uri
        self.fenetre_uri = fenetre_uri
        self.lampe_uri = lampe_uri
        self.ventilation_uri = ventilation_uri
        self.volet_uri = volet_uri
        print(f"  [{self.agent_uri.split('#')[-1]}] Agent Bâtiments (Simulateur Complet) initialisé.")

    def simuler_et_publier_donnees(self, temp_int: float, co2: float, lumi: float, temp_ext: float, 
                                     humi: float, occup: int, lumi_ext: float):
        
        # Ce bloc crée de nouvelles instances de Mesure pour chaque type de donnée
        # et les attache aux capteurs correspondants dans le graphe.
        
        print(f"  [{self.agent_uri.split('#')[-1]}] Simulation: TempInt={temp_int}°C, CO2={co2}ppm, Lumi={lumi}lux, TempExt={temp_ext}°C, Humi={humi}%, Occup={occup}.")
        timestamp = datetime.datetime.now(datetime.timezone.utc)
        
        # --- Données intérieures ---
        # Température intérieure
        obs_temp_uri = EX[f"MesureTemp_{timestamp.strftime('%Y%m%d%H%M%S%f')}"]
        self.graph.add((obs_temp_uri, RDF.type, EX.MesureTemperature)) 
        self.graph.add((obs_temp_uri, EX.aPourValeur, Literal(temp_int, datatype=XSD.float)))
        self.graph.add((obs_temp_uri, EX.timestamp, Literal(timestamp)))
        self.graph.add((self.capteur_temp_uri, EX.aPourTemperatureActuelle, obs_temp_uri))
        
        # CO2
        obs_co2_uri = EX[f"MesureCO2_{timestamp.strftime('%Y%m%d%H%M%S%f')}"]
        self.graph.add((obs_co2_uri, RDF.type, EX.MesureCO2))
        self.graph.add((obs_co2_uri, EX.aPourValeur, Literal(co2, datatype=XSD.float)))
        self.graph.add((obs_co2_uri, EX.timestamp, Literal(timestamp)))
        self.graph.add((self.capteur_co2_uri, EX.aPourCO2Actuel, obs_co2_uri))

        # Luminosité
        obs_lumi_uri = EX[f"MesureLuminosite_{timestamp.strftime('%Y%m%d%H%M%S%f')}"]
        self.graph.add((obs_lumi_uri, RDF.type, EX.MesureLuminosite))
        self.graph.add((obs_lumi_uri, EX.aPourValeur, Literal(lumi, datatype=XSD.float)))
        self.graph.add((obs_lumi_uri, EX.timestamp, Literal(timestamp)))
        self.graph.add((self.capteur_lumi_uri, EX.aPourLuminositeActuelle, obs_lumi_uri))
        
        # AJOUT: Humidité
        obs_humi_uri = EX[f"MesureHumidite_{timestamp.strftime('%Y%m%d%H%M%S%f')}"]
        self.graph.add((obs_humi_uri, RDF.type, EX.MesureHumidite))
        self.graph.add((obs_humi_uri, EX.aPourValeur, Literal(humi, datatype=XSD.float)))
        self.graph.add((obs_humi_uri, EX.timestamp, Literal(timestamp)))
        self.graph.add((self.capteur_humi_uri, EX.aPourHumiditeActuelle, obs_humi_uri))

        # AJOUT: Occupation
        obs_occup_uri = EX[f"MesureOccupation_{timestamp.strftime('%Y%m%d%H%M%S%f')}"]
        self.graph.add((obs_occup_uri, RDF.type, EX.MesureOccupation))
        self.graph.add((obs_occup_uri, EX.aPourValeur, Literal(occup, datatype=XSD.integer)))
        self.graph.add((obs_occup_uri, EX.timestamp, Literal(timestamp)))
        self.graph.add((self.capteur_occup_uri, EX.aPourOccupationActuelle, obs_occup_uri))
        
        # On met aussi à jour la propriété de convenance :estOccupee sur la pièce elle-même
        self.graph.set((self.bureau_uri, EX.estOccupee, Literal(occup > 0, datatype=XSD.boolean)))
        
        # --- Données extérieures ---
        # Température extérieure
        obs_temp_ext_uri = EX[f"MesureTempExt_{timestamp.strftime('%Y%m%d%H%M%S%f')}"]
        self.graph.add((obs_temp_ext_uri, RDF.type, EX.MesureTemperature))
        self.graph.add((obs_temp_ext_uri, EX.aPourValeur, Literal(temp_ext, datatype=XSD.float)))
        self.graph.add((obs_temp_ext_uri, EX.timestamp, Literal(timestamp)))
        self.graph.add((self.capteur_temp_ext_uri, EX.aPourTemperatureActuelle, obs_temp_ext_uri))

        obs_lumi_ext_uri = EX[f"MesureLuminositeExt_{timestamp.strftime('%Y%m%d%H%M%S%f')}"]
        self.graph.add((obs_lumi_ext_uri, RDF.type, EX.MesureLuminosite))
        self.graph.add((obs_lumi_ext_uri, EX.aPourValeur, Literal(lumi_ext, datatype=XSD.float)))
        self.graph.add((obs_lumi_ext_uri, EX.timestamp, Literal(timestamp)))
        self.graph.add((self.capteur_lumi_ext_uri, EX.aPourLuminositeActuelle, obs_lumi_ext_uri))

        
    def simuler_etat_actionneur(self, equipment_uri: URIRef, state_uri: URIRef):
        self.graph.set((equipment_uri, EX.aPourEtat, state_uri))
        print(f"  [{self.agent_uri.split('#')[-1]}] État actionneur simulé: {equipment_uri.split('#')[-1]} = {state_uri.split('#')[-1]}.")


class AgentCalendrier:
    def __init__(self, graph: Graph, agent_uri: URIRef):
        """
        Rôle : Le "visionnaire" du système. Regarde dans le futur.
        - Surveille les événements planifiés (ex: arrivées, réunions) sur le Tableau Blanc.
        - Crée des intentions proactives (ex: "préchauffer la pièce") pour anticiper les besoins.
        """
        self.graph = graph
        self.agent_uri = agent_uri
        print(f"  [{self.agent_uri.split('#')[-1]}] Agent Calendrier initialisé.")

    async def evaluer_et_creer_intentions_proactives(self, prechauffage_window_minutes: int = 90):
        """
        Cherche les événements dans une fenêtre de temps future et publie des intentions.
        """
        print(f"  [{self.agent_uri.split('#')[-1]}] Recherche d'événements proactifs...")

        # 1. Récupère tous les événements du graphe.
        query = """
        SELECT ?event ?heureDebut ?zone ?occupant WHERE {
            ?event a ex:EvenementCalendrier ;
                   ex:heureDebut ?heureDebut ;
                   ex:viseZone ?zone ;
                   ex:concerneOccupant ?occupant .
        }"""
        all_events = self.graph.query(query, initNs=GLOBAL_NAMESPACES)

        # 2. Filtre les événements en Python pour plus de robustesse.
        now = datetime.datetime.now(datetime.timezone.utc)
        future_limit = now + datetime.timedelta(minutes=prechauffage_window_minutes)
        
        upcoming_events = []
        for event in all_events:
            # On s'assure que la date est bien un Literal et on la convertit
            if isinstance(event.heureDebut, Literal):
                event_time = event.heureDebut.toPython()
                # On compare les dates en Python
                if now < event_time < future_limit:
                    upcoming_events.append(event)
        
        if not upcoming_events:
            print(f"  [{self.agent_uri.split('#')[-1]}] Aucun événement pertinent trouvé dans la fenêtre de temps.")
            return

        # 3. Crée une intention pour chaque événement pertinent.
        for event in upcoming_events:
            occupant_prefs = self.graph.value(event.occupant, EX.preferenceTemperature)
            if not occupant_prefs: continue
            
            temp_cible = float(occupant_prefs)
            heure_cible = event.heureDebut.toPython()
            zone_uri = event.zone
            zone_nom = self.graph.value(zone_uri, EX.nomZone) or zone_uri.split('#')[-1]

            description = f"Préchauffer {zone_nom} pour atteindre {temp_cible}°C d'ici {heure_cible.strftime('%H:%M')}."
            
            print(f"  [{self.agent_uri.split('#')[-1]}] Intention proactive créée: '{description}'")

            add_intention(
                graph=self.graph,
                agent_uri=self.agent_uri,
                bureau_uri=zone_uri,
                description=description,
                generic_type="AtteindreObjectifTemporel", 
                concerne_equipement=None
            )



class AgentConforts:
    def __init__(self, graph: Graph, bureau_uri: URIRef, agent_uri: URIRef, occupant_uri: URIRef, llm_manager: LLMManager):
        """
        Rôle : Le "représentant" de l'occupant.
        - Utilise une logique déterministe (fiable et rapide) pour détecter les besoins de confort immédiats.
        - Ne s'active que si la pièce est occupée.
        - Propose des solutions simples et directes pour résoudre les problèmes (ex: "il fait froid -> allumer le chauffage").
        - Peut proposer des solutions synergiques simples (ex: "trop chaud + trop lumineux -> fermer les volets").
        """
        self.graph = graph
        self.bureau_uri = bureau_uri
        self.agent_uri = agent_uri
        self.occupant_uri = occupant_uri
        self.llm = llm_manager 
        print(f"  [{self.agent_uri.split('#')[-1]}] Agent Conforts (Logique Fiable) initialisé.")

        self.equipment_map = {
            "RadiateurBureau1": EX.RadiateurBureau1,
            "FenetreBureau1": EX.FenetreBureau1,
            "LampeBureau1": EX.LampeBureau1,
            "VentilationBureau1": EX.VentilationBureau1,
            "ClimatiseurBureau1": EX.ClimatiseurBureau1,
            "VoletBureau1": EX.VoletBureau1,
            "None": None
        }

    async def _deduire_besoins_fiables(self):
        """
        [VERSION FINALE] Calcule les besoins de manière 100% fiable en utilisant Python.
        C'est le "Système 1" (réflexe) de notre architecture.
        """
        print(f"  [{self.agent_uri.split('#')[-1]}] Détection fiable des besoins (logique Python)...")
        
        #1 : Collecte des faits
        temp_actuelle = get_current_sensor_value(self.graph, EX.CapteurTempBureau1, "aPourTemperatureActuelle")
        pref_temp = get_occupant_preference(self.graph, self.occupant_uri, "preferenceTemperature")
        co2_actuel = get_current_sensor_value(self.graph, EX.CapteurCO2Bureau1, "aPourCO2Actuel")
        pref_co2 = get_occupant_preference(self.graph, self.occupant_uri, "preferenceCO2")
        lumi_actuelle = get_current_sensor_value(self.graph, EX.CapteurLuminositeBureau1, "aPourLuminositeActuelle")
        pref_lumi = get_occupant_preference(self.graph, self.occupant_uri, "preferenceLuminosite")
        # ÉTAPE CLÉ : On vérifie l'occupation d'abord
        occupants = get_current_sensor_value(self.graph, EX.CapteurOccupationBureau1, "aPourOccupationActuelle")
    
        if not occupants or occupants == 0:
            print("Pièce inoccupée, aucun besoin de confort à calculer.")
            return {"needs": [], "intentions": []}

        #2 : Calcul déterministe des besoins
        needs = []
        if temp_actuelle is not None and pref_temp is not None:
            if temp_actuelle < pref_temp - 1.0: # Seuil pour le chauffage
                needs.append({"type": EX.BesoinChauffage, "description": f"Température ({temp_actuelle}°C) trop basse."})
            elif temp_actuelle > pref_temp + 1.0: # Seuil pour le rafraîchissement
                needs.append({"type": EX.BesoinRafraichissement, "description": f"Température ({temp_actuelle}°C) trop élevée."})

        if co2_actuel is not None and pref_co2 is not None and co2_actuel > pref_co2:
            needs.append({"type": EX.BesoinVentilation, "description": f"CO2 ({co2_actuel}ppm) trop élevé."})
        
        if lumi_actuelle is not None and pref_lumi is not None and lumi_actuelle < pref_lumi - 50:
            needs.append({"type": EX.BesoinLuminosite, "description": f"Luminosité ({lumi_actuelle} lux) trop faible."})

        elif lumi_actuelle is not None and pref_lumi is not None and lumi_actuelle > pref_lumi + 500: # Seuil pour l'éblouissement
            # Assure-toi que la classe :BesoinReductionLuminosite existe bien dans ton ontologie
            needs.append({"type": EX.BesoinReductionLuminosite, "description": f"Luminosité ({lumi_actuelle} lux) trop élevée (éblouissement)." })


        #3 : Génération d'intentions simples par défaut
        # L'AgentStratege aura la charge de les améliorer ou de les remplacer.
        intentions = []
        
        # Logique pour le chauffage 
        if any(n['type'] == EX.BesoinChauffage for n in needs):
            intentions.append({"type": "AugmenterTemperature", "description": "Activer le radiateur pour le confort thermique.", "equipment": "RadiateurBureau1"})
        
        #Logique pour le rafraîchissement
        if any(n['type'] == EX.BesoinRafraichissement for n in needs):
            # On ajoute une condition intelligente : s'il y a aussi trop de lumière, on ferme les volets !
            if any(n['type'] == EX.BesoinReductionLuminosite for n in needs):
                intentions.append({"type": "FermerVolet", "description": "Fermer les volets pour réduire la chaleur et l'éblouissement.", "equipment": "VoletBureau1"})
            else:
                intentions.append({"type": "DiminuerTemperature", "description": "Activer la climatisation pour rafraîchir la pièce.", "equipment": "ClimatiseurBureau1"})
        # Logique pour la luminosité seule
        elif any(n['type'] == EX.BesoinReductionLuminosite for n in needs):
            intentions.append({"type": "FermerVolet", "description": "Fermer les volets pour réduire l'éblouissement.", "equipment": "VoletBureau1"})
    
        #Logique pour la ventilation
        if any(n['type'] == EX.BesoinVentilation for n in needs):
            intentions.append({"type": "AmeliorerQualiteAir", "description": "Activer la ventilation mécanique pour améliorer la qualité de l'air.", "equipment": "VentilationBureau1"})
            
        #Logique pour la luminosité
        if any(n['type'] == EX.BesoinLuminosite for n in needs):
            intentions.append({"type": "AugmenterLuminosite", "description": "Allumer la lampe pour améliorer le confort visuel.", "equipment": "LampeBureau1"})
        
        return {"needs": needs, "intentions": intentions}

    async def evaluer_et_publier(self):
        """Cycle d'évaluation et de publication des besoins et intentions de base."""
        clear_previous_intentions_and_conflicts_by_agent(self.graph, self.agent_uri, self.bureau_uri)
        deductions = await self._deduire_besoins_fiables()
        
        # Publication des besoins détectés
        if deductions["needs"]:
            print(f"  [{self.agent_uri.split('#')[-1]}] Besoins détectés: {[n['type'].split('#')[-1] for n in deductions['needs']]}")
        for need_data in deductions["needs"]:
            add_need_to_piece(self.graph, self.agent_uri, self.bureau_uri, need_data['type'], need_data['description'])

        # Publication des intentions de base
        for intention_data in deductions["intentions"]:
            add_intention(self.graph, self.agent_uri, self.bureau_uri,
                          intention_data["description"], intention_data["type"],
                          self.equipment_map.get(intention_data["equipment"]))
        
        # Sauvegarde
        try:
            self.graph.serialize(destination=DYNAMIC_WHITEBOARD_FILE, format="turtle")
        except Exception as e:
            print(f"  [{self.agent_uri.split('#')[-1]}] ERREUR lors de la sauvegarde du Tableau Blanc : {e}")
            
        
class AgentEcoEnergie:
    """
    Rôle : Le "gardien" de l'efficacité énergétique.
    - Mission 1 (prioritaire) : Détecter les conflits énergétiques. Il surveille les intentions
      proposées par d'autres agents et s'y oppose (via une Objection) s'il existe
      une solution plus économique.
    - Mission 2 (secondaire) : S'il n'y a pas de conflit, il cherche proactivement des
      opportunités de gaspillage non détectées (ex: une lumière restée allumée).
    """
    def __init__(self, graph: Graph, bureau_uri: URIRef, agent_uri: URIRef, llm_manager: LLMManager):
        self.graph = graph
        self.bureau_uri = bureau_uri
        self.agent_uri = agent_uri
        self.llm = llm_manager
        print(f"  [{self.agent_uri.split('#')[-1]}] Agent EcoEnergie initialisé pour {bureau_uri.split('#')[-1]}.")

        self.equipment_map = {
            "LampeBureau1": EX.LampeBureau1,
            "FenetreBureau1": EX.FenetreBureau1,
            "VentilationBureau1": EX.VentilationBureau1
        }

    async def _deduire_intentions_proactives(self):
        """
        Logique interne où l'agent cherche de nouvelles opportunités d'économie d'énergie, uniquement si aucune objection n'est prioritaire.
        """
        print(f"  [{self.agent_uri.split('#')[-1]}] Recherche proactive d'économies d'énergie (LLM)...")

        # Récupération des données 
        lumi_actuelle = get_current_sensor_value(self.graph, EX.CapteurLuminositeBureau1, "aPourLuminositeActuelle")
        pref_lumi = get_occupant_preference(self.graph, EX.OccupantJean, "preferenceLuminosite")
        eclairage_etat = get_equipment_state(self.graph, EX.LampeBureau1)
        chauffage_etat = get_equipment_state(self.graph, EX.RadiateurBureau1)
        fenetre_etat = get_equipment_state(self.graph, EX.FenetreBureau1)

        if any(v is None for v in [lumi_actuelle, pref_lumi, eclairage_etat, chauffage_etat, fenetre_etat]):
            print(f"  [{self.agent_uri.split('#')[-1]}] Données manquantes, annulation de la déduction proactive.")
            return []

        # Calcul des drapeaux booléens 
        is_lumi_sufficient_and_eclairage_active = (pref_lumi - 50.0 <= lumi_actuelle <= pref_lumi + 50.0) and eclairage_etat == "Allume"
        is_chauffage_active_and_fenetre_ouverte = chauffage_etat == "Actif" and fenetre_etat == "Ouverte"

        system_prompt_content = """
Vous êtes l'Agent EcoEnergie. Votre rôle est de proposer des intentions d'action pour économiser l'énergie.
Répondez uniquement avec une liste de mots-clés séparés par des virgules.
Si aucune déduction n'est faite, répondez par `AUCUNE_DEDUCTION_ENERGIE`.
Mots-clés possibles: `BESOIN_ECONOMIE_ENERGIE`, `INTENTION_ETEINDRE_ECLAIRAGE_ECONOMIE`, `INTENTION_FERMER_FENETRE_ECONOMIE`
"""
        human_prompt_content = f"""
État actuel :
- Luminosité: {lumi_actuelle}lux (Préférence: {pref_lumi}lux), État Lampe: {eclairage_etat}
- État Radiateur: {chauffage_etat}, État Fenêtre: {fenetre_etat}

Conditions (VRAI/FAUX):
- IS_LUMI_SUFFICIENT_AND_ECLAIRAGE_ACTIVE: {is_lumi_sufficient_and_eclairage_active}
- IS_CHAUFFAGE_ACTIVE_AND_FENETRE_OUVERTE: {is_chauffage_active_and_fenetre_ouverte}

Instructions:
SI IS_LUMI_SUFFICIENT_AND_ECLAIRAGE_ACTIVE est VRAI, générez: BESOIN_ECONOMIE_ENERGIE, INTENTION_ETEINDRE_ECLAIRAGE_ECONOMIE
SINON SI IS_CHAUFFAGE_ACTIVE_AND_FENETRE_OUVERTE est VRAI, générez: BESOIN_ECONOMIE_ENERGIE, INTENTION_FERMER_FENETRE_ECONOMIE
SINON, générez: AUCUNE_DEDUCTION_ENERGIE
"""
        prompt_data_for_llm = {"system_prompt_content": system_prompt_content, "human_prompt_content": human_prompt_content}

        try:
            response_content = await self.llm.ainvoke(prompt_data_for_llm)
            if not response_content:
                print(f"  [{self.agent_uri.split('#')[-1]}]L'appel au LLM n'a retourné aucune réponse. Annulation de la déduction.")
                return [] # On retourne une liste vide pour ne pas causer d'erreur
            raw_response_str = response_content.strip()
            print(f"  [{self.agent_uri.split('#')[-1]}] Réponse brute du LLM: {raw_response_str}")
            
            intentions = []
            keywords = [k.strip() for k in raw_response_str.split(',') if k.strip()]
            
            if "INTENTION_ETEINDRE_ECLAIRAGE_ECONOMIE" in keywords:
                intentions.append({"type": "DiminuerLuminosite", "description": "Éteindre la lampe pour économiser l'énergie.", "equipment": "LampeBureau1"})
            if "INTENTION_FERMER_FENETRE_ECONOMIE" in keywords:
                intentions.append({"type": "DiminuerTemperature", "description": "Fermer la fenêtre pour éviter la perte de chaleur.", "equipment": "FenetreBureau1"})
            
            return intentions
        except Exception as e:
            print(f"  [{self.agent_uri.split('#')[-1]}] ERREUR lors de la déduction proactive: {e}")
            return []

    async def evaluer_et_publier(self):
        """
        [VERSION FINALE - Logique Complète]
        L'agent suit ses deux missions par ordre de priorité.
        """
        print(f"  [{self.agent_uri.split('#')[-1]}] Évaluation des opportunités d'économie d'énergie...")
        clear_previous_intentions_and_conflicts_by_agent(self.graph, self.agent_uri, self.bureau_uri)

        # MISSION 1 : Arrêter le Gaspillage (Négociation) 
        chauffage_etat = get_equipment_state(self.graph, EX.RadiateurBureau1)
        current_intentions = get_all_current_intentions(self.graph, self.bureau_uri)
        a_objecte = False

        for intent in current_intentions:
            intent_type_uri = self.graph.value(subject=intent['uri'], predicate=RDF.type)
            
            # Règle 1 : Conflit "Fenêtre vs. Chauffage"
            chauffage_etat = get_equipment_state(self.graph, EX.RadiateurBureau1)
            if intent_type_uri == EX.OuvrirFenetre and chauffage_etat == "Actif":
                print("Conflit détecté ! Chauffage actif et intention d'ouvrir la fenêtre.")
                add_objection(
                    graph=self.graph, agent_uri=self.agent_uri, bureau_uri=self.bureau_uri,
                    original_intention_uri=intent['uri'],
                    objection_reason="Gaspillage d'énergie car le chauffage est actif.",
                    contre_proposition_desc="Activer la ventilation mécanique pour renouveler l'air sans perte de chaleur.",
                    contre_proposition_type="AmeliorerQualiteAir",
                    contre_proposition_equip=EX.VentilationBureau1
                )
                a_objecte = True
                break

            # Règle 2 : Conflit "Climatiseur vs. Fraîcheur extérieure"
            elif intent_type_uri == EX.ActiverClimatiseur:
                temp_ext = get_current_sensor_value(self.graph, EX.CapteurTemperatureExterieur, "aPourTemperatureActuelle")
                temp_int = get_current_sensor_value(self.graph, EX.CapteurTempBureau1, "aPourTemperatureActuelle")
                
                if temp_ext is not None and temp_int is not None and temp_ext < temp_int:
                    print("Opportunité détectée ! Contestation de l'usage du climatiseur.")
                    add_objection(
                        graph=self.graph, agent_uri=self.agent_uri, bureau_uri=self.bureau_uri,
                        original_intention_uri=intent['uri'],
                        objection_reason="Gaspillage d'énergie : il est plus économique d'ouvrir la fenêtre car il fait plus frais dehors.",
                        contre_proposition_desc="Ouvrir la fenêtre pour un rafraîchissement naturel et gratuit.",
                        contre_proposition_type="DiminuerTemperature",
                        contre_proposition_equip=EX.FenetreBureau1
                    )
                    a_objecte = True
                    break
        
        if a_objecte:
            self.graph.serialize(destination=DYNAMIC_WHITEBOARD_FILE, format="turtle")
            return

        # MISSION 2 : Proposer des Économies (Action Proactive)
        print(f"  [{self.agent_uri.split('#')[-1]}] MISSION 2 : Aucun gaspillage à corriger. Recherche de nouvelles économies...")
        proactive_intentions = await self._deduire_intentions_proactives()
        
        for intention_data in proactive_intentions:
            add_intention(
                self.graph, self.agent_uri, self.bureau_uri,
                intention_data["description"], intention_data["type"],
                self.equipment_map.get(intention_data["equipment"])
            )
        
        self.graph.serialize(destination=DYNAMIC_WHITEBOARD_FILE, format="turtle")


class AgentSimulation:
    """
    Agent de simulation avec un modèle physique plus réaliste
    intégrant l'inertie thermique des murs et les apports de chaleur dynamiques.
    """
    def __init__(self, volume_piece: float, 
                 capacite_thermique_air: float, 
                 capacite_thermique_murs: float,
                 resistance_thermique_murs: float,
                 surface_vitree_m2: float,       
                 facteur_solaire: float = 0.6):  
        # Paramètres thermiques
        self.C_air = capacite_thermique_air
        self.C_mur = capacite_thermique_murs
        self.R_wall = resistance_thermique_murs
        
        # Paramètres solaires
        self.surface_vitree_m2 = surface_vitree_m2
        self.facteur_solaire = facteur_solaire # (g-value)

        # Paramètres internes
        self.Volume = volume_piece
        self.CO2_prod_par_personne = 0.005 # L/s/personne, valeur plus standard
        self.vapeur_par_personne = 0.015 # g/s/personne
        print(f"  [Système] AgentSimulation (modèle avancé) initialisé.")

    def run_simulation_and_get_results(self, 
                                       # Conditions initiales
                                       T_air_initiale: float, T_mur_initiale: float, 
                                       H_initiale: float, CO2_initial: float,
                                       # Conditions extérieures
                                       T_ext: float, lumi_ext_lux: float,
                                       # Actions et consignes
                                       occupants: int = 1,
                                       puissance_chauffage: float = 0,
                                       puissance_apports_internes: float = 0, 
                                       position_volet: float = 0.0, # 0.0=ouvert, 1.0=fermé
                                       duree_heures: int = 1, 
                                       consigne_temp: float = 21.0):
        
        print(f"  [AgentSimulation] Lancement simulation avancée...")
        
        dt = 60 # Time step en secondes
        temps_total = duree_heures * 3600
        N = int(temps_total / dt)

        T_air, T_mur, CO2, energie = T_air_initiale, T_mur_initiale, CO2_initial, 0.0
        temps_pour_consigne = None

        # Conversion approximative Lux -> W/m² (très variable, mais c'est une base)
        irradiance_solaire_W_m2 = lumi_ext_lux / 120 

        for step in range(N):
            # Calcul de l'apport solaire dynamique
            gain_solaire_W = irradiance_solaire_W_m2 * self.surface_vitree_m2 * self.facteur_solaire * (1 - position_volet)
            
            # Échanges thermiques (simplifiés)
            flux_mur_vers_ext = (T_mur - T_ext) / self.R_wall
            flux_air_vers_mur = (T_air - T_mur) / (self.R_wall / 10) # R_convection interne
            
            # Mise à jour de la température des murs (stockage/restitution)
            T_mur += ((flux_air_vers_mur - flux_mur_vers_ext) / self.C_mur) * dt
            
            # Mise à jour de la température de l'air
            apports_totaux_air = puissance_chauffage + gain_solaire_W + puissance_apports_internes
            T_air += ((apports_totaux_air - flux_air_vers_mur) / self.C_air) * dt
            
            # CO2 (le modèle reste similaire)
            CO2 += ((400 - CO2) * 0.0 + (occupants * self.CO2_prod_par_personne * 1000 / self.Volume)) * dt

            # Calcul de l'énergie
            energie += puissance_chauffage * dt / 3600000.0 # Conversion Ws -> kWh

            if T_air >= consigne_temp and temps_pour_consigne is None:
                temps_pour_consigne = (step * dt) / 60
        
        results = {
            "temperature_finale_air": round(T_air, 1),
            "temperature_finale_murs": round(T_mur, 1),
            "co2_final": round(CO2, 0),
            "energie_consommee_kWh": round(energie, 3),
            "temps_pour_consigne_min": round(temps_pour_consigne, 1) if temps_pour_consigne is not None else float('inf')
        }
        print(f"  [AgentSimulation] Fin simulation. Résultats: {results}")
        return results
        

class AgentMediateur:
    """
    Rôle : L'"arbitre" des conflits.
    - Se déclenche uniquement si une "Objection" est publiée sur le Tableau Blanc.
    - Enrichit le contexte du débat (en ajoutant des données comme la température extérieure).
    - Fait appel à un LLM pour arbitrer le conflit entre l'intention initiale et la contre-proposition.
    - Met à jour le Tableau Blanc avec la décision finale.
    """
    
    def __init__(self, graph: Graph, agent_uri: URIRef, llm_manager: LLMManager):
        self.graph = graph
        self.agent_uri = agent_uri
        self.llm = llm_manager
        print(f"  [{self.agent_uri.split('#')[-1]}] Agent Médiateur initialisé.")

    def _get_unresolved_objections(self):
        """
        Trouve les objections ciblant des intentions qui n'ont pas encore été traitées.
        C'est le "déclencheur" de l'agent.
        """
        query = """
        SELECT ?objection ?reason ?original_intention ?contre_proposition WHERE {
            ?objection rdf:type ex:Objection .
            ?objection ex:estUneObjectionA ?original_intention .
            
            # LE DÉCLENCHEUR : On ne traite que les intentions fraîchement proposées.
            ?original_intention ex:aPourStatut ex:Statut_Proposee .
            
            ?objection ex:proposeAlternative ?contre_proposition .
            OPTIONAL { ?objection rdfs:comment ?reason . }
        }
        """
        return self.graph.query(query, initNs=GLOBAL_NAMESPACES)

    async def evaluer_et_arbitrer(self):
        """
        Le cycle de vie de l'agent : chercher des objections et les arbitrer.
        """
        print(f"  [{self.agent_uri.split('#')[-1]}] Recherche d'objections à arbitrer...")
        objections_a_traiter = self._get_unresolved_objections()
    
        a_traite_une_objection = False
        for objection_data in list(objections_a_traiter):
            a_traite_une_objection = True
            print(f"  [{self.agent_uri.split('#')[-1]}]Objection trouvée ! Arbitrage en cours...")
            
            
            
            try:
                # --- ENRICHISSEMENT DU CONTEXTE ---
                original_intention = objection_data.original_intention
                contre_proposition = objection_data.contre_proposition

                temp_int = get_current_sensor_value(self.graph, EX.CapteurTempBureau1, "aPourTemperatureActuelle")
                temp_ext = get_current_sensor_value(self.graph, EX.CapteurTemperatureExterieur, "aPourTemperatureActuelle")
                
                contexte_string = f"Contexte actuel : Température intérieure = {temp_int}°C, Température extérieure = {temp_ext}°C."
                
                # --- NOUVEAU PROMPT ENRICHI ---
                desc_originale = self.graph.value(objection_data.original_intention, EX.description)
                desc_contre_prop = self.graph.value(objection_data.contre_proposition, EX.description)
                
                system_prompt = "Tu es un médiateur expert en énergie. Choisis la solution la plus logique et la plus économique. Réponds UNIQUEMENT par `OPTION_1` ou `OPTION_2`."
                human_prompt = (f"{contexte_string}\n\n"
                            f"Débat à arbitrer:\n"
                            f"- OPTION_1 (Action initiale): \"{desc_originale}\"\n"
                            f"- OPTION_2 (Contre-proposition économique): \"{desc_contre_prop}\"\n\n"
                            f"Raison de l'objection: {objection_data.reason}\n\n"
                            f"Quelle option est la plus intelligente dans ce contexte ?")
                llm_response = await self.llm.ainvoke({"system_prompt_content": system_prompt, "human_prompt_content": human_prompt})
                decision = re.findall(r"OPTION_(\d+)", llm_response)
            
                if decision and decision[-1] == "2":
                    print(f"  [{self.agent_uri.split('#')[-1]}] DÉCISION: Contre-proposition ACCEPTÉE.")
                    # Supprime l'intention initiale
                    self.graph.update(f"DELETE WHERE {{ <{original_intention}> ?p ?o . }}")
                    # Valide la CP
                    self.graph.remove((contre_proposition, EX.aPourStatut, EX.Statut_Proposee))
                    self.graph.add((contre_proposition, EX.aPourStatut, EX.Statut_Validee))
            
                    # Journal de résolution
                    add_resolution_log(
                        graph=self.graph,
                        agent_uri=self.agent_uri,
                        bureau_uri=EX.Bureau1,
                        resolution_description="Contre-proposition retenue (plus économe).",
                        conflict_uri=EX[f"Conflit_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"],  # si tu ne crées pas de Conflit séparé
                        validated_intention_uri=contre_proposition,
                        discarded_intentions_uris=[original_intention]
                    )
            
                else:
                    print(f"  [{self.agent_uri.split('#')[-1]}] DÉCISION: Contre-proposition REJETÉE.")
                    # Supprime la CP
                    self.graph.update(f"DELETE WHERE {{ <{contre_proposition}> ?p ?o . }}")
                    # Valide l'intention initiale
                    self.graph.remove((original_intention, EX.aPourStatut, EX.Statut_Proposee))
                    self.graph.add((original_intention, EX.aPourStatut, EX.Statut_Validee))
            
                    # Journal de résolution
                    add_resolution_log(
                        graph=self.graph,
                        agent_uri=self.agent_uri,
                        bureau_uri=EX.Bureau1,
                        resolution_description="Intention initiale retenue (CP rejetée).",
                        conflict_uri=EX[f"Conflit_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"],
                        validated_intention_uri=original_intention,
                        discarded_intentions_uris=[contre_proposition]
                    )
            
                # Supprime l’objection (une seule fois)
                self.graph.update(f"DELETE WHERE {{ <{objection_data.objection}> ?p ?o . }}")
                print(f"  [{self.agent_uri.split('#')[-1]}] Objection '{objection_data.objection.split('#')[-1]}' traitée et supprimée.")
            
            except Exception as e:
                print(f"  [{self.agent_uri.split('#')[-1]}] ERREUR Médiateur LLM: {e}.")

class AgentStratege:
    """
    Rôle : Le "stratège" pour les problèmes complexes.
    - Se déclenche uniquement si plusieurs besoins de confort sont détectés en même temps.
    - Construit un "briefing" complet de la situation (contexte, besoins, actions possibles et leurs synergies).
    - Fait appel à un LLM pour trouver l'UNIQUE action qui résout le plus de problèmes simultanément.
    - Annule les intentions de base et les remplace par sa solution optimisée.
    """
    
    def __init__(self, graph: Graph, agent_uri: URIRef, llm_manager: LLMManager):
        self.graph = graph
        self.agent_uri = agent_uri
        self.llm = llm_manager
        print(f"  [{self.agent_uri.split('#')[-1]}] Agent Stratège (LLM-driven) initialisé.")

    def _build_strategic_briefing(self):
        """
        Construit un dossier stratégique complet pour le LLM en interrogeant le graphe.
        """
        # 1. Collecter le contexte actuel
        context = {
            # Données des capteurs 
            "temp_int": get_current_sensor_value(self.graph, EX.CapteurTempBureau1, "aPourTemperatureActuelle"),
            "temp_ext": get_current_sensor_value(self.graph, EX.CapteurTemperatureExterieur, "aPourTemperatureActuelle"),
            "co2": get_current_sensor_value(self.graph, EX.CapteurCO2Bureau1, "aPourCO2Actuel"),
            "humi_int": get_current_sensor_value(self.graph, EX.CapteurHumiditeBureau1, "aPourHumiditeActuelle"),
            "lumi_int": get_current_sensor_value(self.graph, EX.CapteurLuminositeBureau1, "aPourLuminositeActuelle"),
            "occupants": int(get_current_sensor_value(self.graph, EX.CapteurOccupationBureau1, "aPourOccupationActuelle") or 0),
            
            # États des équipements 
            "etat_radiateur": get_equipment_state(self.graph, EX.RadiateurBureau1),
            "etat_fenetre": get_equipment_state(self.graph, EX.FenetreBureau1),
            "etat_ventilation": get_equipment_state(self.graph, EX.VentilationBureau1),
            "etat_lampe": get_equipment_state(self.graph, EX.LampeBureau1),
            "etat_volet": get_equipment_state(self.graph, EX.VoletBureau1),
            "etat_clim": get_equipment_state(self.graph, EX.ClimatiseurBureau1)
        }

        # 2. Collecter les besoins actifs
        query_besoins = "SELECT DISTINCT ?typeBesoin WHERE { ?bureau ex:aPourBesoin ?b . ?b a ?typeBesoin . }"
        besoins_res = self.graph.query(query_besoins, initNs=GLOBAL_NAMESPACES, initBindings={'bureau': EX.Bureau1})
        besoins = {res.typeBesoin.split('#')[-1] for res in besoins_res}

        # 3. Collecter les actions candidates et leurs effets (directs + synergiques)
        actions = {}
        query_actions = """
        SELECT DISTINCT ?action ?beneficeLabel WHERE {
            ?action rdfs:subClassOf ex:Intention .
            OPTIONAL {
                ?action rdfs:subClassOf [
                    rdf:type owl:Restriction ;
                    owl:onProperty ex:aPourBeneficeAdditionnel ;
                    owl:someValuesFrom ?benefice
                ] .
                ?benefice rdfs:label ?beneficeLabel .
            }
        }"""
        actions_res = self.graph.query(query_actions, initNs=GLOBAL_NAMESPACES)
        
        for row in actions_res:
            action_name = row.action.split('#')[-1]
            if action_name not in actions:
                actions[action_name] = {"effets": set()}
            if row.beneficeLabel:
                actions[action_name]["effets"].add(str(row.beneficeLabel))
        
        # Ajoute manuellement les effets principaux pour la clarté du prompt
        if "ActiverChauffage" in actions: actions["ActiverChauffage"]["effets"].add("Augmentation de la Température")
        if "DesactiverChauffage" in actions: actions["DesactiverChauffage"]["effets"].add("Économie d'Énergie (Chauffage)")
        
        if "OuvrirFenetre" in actions:
            actions["OuvrirFenetre"]["effets"].add("Rafraîchissement")
            actions["OuvrirFenetre"]["effets"].add("Ventilation")
        if "FermerFenetre" in actions: actions["FermerFenetre"]["effets"].add("Isolation Thermique/Sonore")

        if "AllumerEclairage" in actions: actions["AllumerEclairage"]["effets"].add("Augmentation de la Luminosité")
        if "EteindreEclairage" in actions: actions["EteindreEclairage"]["effets"].add("Réduction de la Luminosité")

        if "OuvrirVolet" in actions: actions["OuvrirVolet"]["effets"].add("Augmentation de la Luminosité Naturelle")
        if "FermerVolet" in actions: actions["FermerVolet"]["effets"].add("Réduction de la Luminosité Naturelle")

        if "ActiverVentilation" in actions: actions["ActiverVentilation"]["effets"].add("Ventilation")
        if "DesactiverVentilation" in actions: actions["DesactiverVentilation"]["effets"].add("Économie d'Énergie (Ventilation)")


        return {
            "contexte": context,
            "besoins": list(besoins),
            "actions": actions
        }

    async def evaluer_et_optimiser(self):
        print(f"  [{self.agent_uri.split('#')[-1]}] Lancement de l'analyse stratégique...")
        
        # On ne lance l'analyse que s'il y a plusieurs besoins à satisfaire
        briefing = self._build_strategic_briefing()
        if len(briefing["besoins"]) < 2:
            print(f"  [{self.agent_uri.split('#')[-1]}] Moins de 2 besoins détectés, pas de synergie complexe à chercher.")
            # On passe la main au Planificateur en promouvant les intentions existantes
            self.graph.update("""
                DELETE { ?i ex:aPourStatut ?s . }
                INSERT { ?i ex:aPourStatut ex:Statut_Optimisee . }
                WHERE { ?i ex:aPourStatut ?s . FILTER(?s IN (ex:Statut_Proposee, ex:Statut_Validee)) }
            """, initNs=GLOBAL_NAMESPACES)
            return

        # Construction du Prompt Stratégique
        system_prompt = "Vous êtes l'Agent Stratège d'un bâtiment intelligent. Votre mission est d'analyser une situation et de choisir l'UNIQUE action la plus efficace qui résout le plus de problèmes simultanément, tout en respectant les contraintes du contexte. Répondez OBLIGATOIREMENT en format JSON."
        
        human_prompt = f"""
# BRIEFING STRATÉGIQUE

## CONTEXTE ACTUEL
- Température Intérieure: {briefing['contexte']['temp_int']}°C
- Température Extérieure: {briefing['contexte']['temp_ext']}°C
- Taux de CO2: {briefing['contexte']['co2']} ppm

## BESOINS À SATISFAIRE
{', '.join(briefing['besoins'])}

## ACTIONS POSSIBLES ET LEURS EFFETS
"""
        for name, data in briefing["actions"].items():
            human_prompt += f"- Action: {name}\n  - Effets: {', '.join(data['effets']) or 'Aucun'}\n"

        human_prompt += """
# MISSION
Analysez les informations ci-dessus. Tenez compte des contraintes logiques (ex: on ne rafraîchit pas en ouvrant la fenêtre s'il fait plus chaud dehors).
Quelle est l'action la plus "rentable" qui résout le plus de besoins de manière valide ?

Répondez avec le JSON suivant, en choisissant une action parmi la liste des actions possibles :
{
  "meilleure_action": "NomDeLAction",
  "equipement_cible": "NomDeLInstanceSpecifiqueDeLEquipement (ex: RadiateurBureau1, FenetreBureau1)",
  "description": "Description de l'action choisie.",
  "type_generique": "TypeGeneriquePourAddIntention"
}
"""
        try:
            llm_response = await self.llm.ainvoke({"system_prompt_content": system_prompt, "human_prompt_content": human_prompt}, json_output=True)
            
            if llm_response and llm_response.get("meilleure_action"):
                decision = llm_response
                print(f"  [{self.agent_uri.split('#')[-1]}] DÉCISION STRATÉGIQUE DU LLM: '{decision['description']}'")
                
                # On nettoie toutes les intentions précédentes proposées par les agents réactifs
                cleanup_query = """
                    DELETE { ?i ?p ?o . } 
                    WHERE { 
                        ?i a ?type .
                        ?type rdfs:subClassOf* ex:Intention . # On cherche toutes les sous-classes
                        
                        ?i ex:aPourStatut ?s .
                        FILTER(?s IN (ex:Statut_Proposee, ex:Statut_Validee))
                        ?i ?p ?o .
                    }"""
                self.graph.update(cleanup_query, initNs=GLOBAL_NAMESPACES)
                
                print(f"  [{self.agent_uri.split('#')[-1]}] Annulation des intentions réactives précédentes.")

                # On crée l'unique intention stratégique
                new_intention_uri = add_intention(
                    self.graph, self.agent_uri, EX.Bureau1,
                    decision["description"],
                    decision["type_generique"],
                    EX[decision["equipement_cible"]]
                )
                
                # On la promeut directement pour le planificateur
                self.graph.remove((new_intention_uri, EX.aPourStatut, EX.Statut_Proposee))
                self.graph.add((new_intention_uri, EX.aPourStatut, EX.Statut_Optimisee))

        except Exception as e:
            print(f"  [{self.agent_uri.split('#')[-1]}] ERREUR Stratège LLM: {e}")
            
class AgentPlanificateur:
    """
    Rôle : L'"ingénieur" du système.
    - Prend les intentions validées (par le Médiateur ou le Stratège) et les transforme en plans concrets.
    - Pour les tâches proactives (ex: préchauffage), il fait appel à l'AgentSimulation pour calculer
      le plan optimal (ex: l'heure exacte pour démarrer le chauffage).
    - Met à jour le statut de l'intention à "Planifiée" une fois son travail terminé.
    """
    
    def __init__(self, graph: Graph, agent_uri: URIRef, llm_manager: LLMManager, simulation_agent: AgentSimulation):
        self.graph = graph
        self.agent_uri = agent_uri
        self.llm = llm_manager
        self.simulation_agent = simulation_agent # Il a besoin du simulateur pour travailler
        print(f"  [{self.agent_uri.split('#')[-1]}] Agent Planificateur initialisé.")

    async def evaluer_et_planifier(self):
        print(f"  [{self.agent_uri.split('#')[-1]}] Recherche d'intentions à planifier...")
        
        # La requête est modifiée pour inclure les intentions "Proposee" (pour le préchauffage)
        get_intentions_query = """
        SELECT ?intention ?type ?description WHERE {
            ?intention ex:aPourStatut ?status .
            FILTER(?status IN (ex:Statut_Optimisee, ex:Statut_Proposee, ex:Statut_Validee))
            ?intention rdf:type ?type .
            ?intention ex:description ?description .
        }"""
        intentions_a_planifier = list(self.graph.query(get_intentions_query, initNs=GLOBAL_NAMESPACES))
        
        if not intentions_a_planifier:
            print(f"  [{self.agent_uri.split('#')[-1]}] Aucune intention à planifier pour le moment.")
            return
    
        for intent in intentions_a_planifier:
            intent_uri = intent.intention
            intent_type = intent.type
            description = str(intent.description)
            
            # --- CAS 1 : Intention proactive de préchauffage ---
            if "Préchauffer" in description:
                    print(f"  [{self.agent_uri.split('#')[-1]}] Traitement d'une intention de préchauffage...")
                    
                    # 1. On extrait les objectifs de la description
                    match = re.search(r'atteindre (\d+\.?\d*)°C d\'ici (\d{2}:\d{2})', description)
                    if not match: continue
                    
                    temp_cible = float(match.group(1))
                    heure_cible_str = match.group(2)
                    
                    # On convertit l'heure cible en vrai objet datetime
                    now = datetime.datetime.now(datetime.timezone.utc)
                    heure_cible = now.replace(hour=int(heure_cible_str.split(':')[0]), minute=int(heure_cible_str.split(':')[1]), second=0, microsecond=0)
    
                    # 2. On utilise la simulation pour estimer le temps de chauffe
                    temp_int = get_current_sensor_value(self.graph, EX.CapteurTempBureau1, "aPourTemperatureActuelle")
                    temp_ext = get_current_sensor_value(self.graph, EX.CapteurTemperatureExterieur, "aPourTemperatureActuelle")
                    humi_int = get_current_sensor_value(self.graph, EX.CapteurHumiditeBureau1, "aPourHumiditeActuelle")
                    co2_int = get_current_sensor_value(self.graph, EX.CapteurCO2Bureau1, "aPourCO2Actuel")
                    lumi_ext = get_current_sensor_value(self.graph, EX.CapteurLuminositeExterieur, "aPourLuminositeActuelle")
                    occupants = int(get_current_sensor_value(self.graph, EX.CapteurOccupationBureau1, "aPourOccupationActuelle") or 0)
                    etat_volet = get_equipment_state(self.graph, EX.VoletBureau1)
                    position_volet = 1.0 if etat_volet == "Fermee" else 0.0 # 1.0 = 100% fermé
                    if not all(v is not None for v in [temp_int, temp_ext, humi_int, co2_int, lumi_ext]):
                        print(f"  [{self.agent_uri.split('#')[-1]}] Données manquantes pour la simulation de préchauffage. Report.")
                        continue
                    apports_internes_W = (occupants * 100) # Apport par personne (chaleur corporelle)
                    apports_internes_W += (occupants * 150) # Apport par équipement (ex: ordinateur)
                    if get_equipment_state(self.graph, EX.LampeBureau1) == "Allume":
                        apports_internes_W += 15 # Apport de la lampe
    
                    if temp_int and temp_int >= temp_cible:
                        print(f"  [{self.agent_uri.split('#')[-1]}] Objectif déjà atteint. Annulation du préchauffage.")
                        self.graph.update(f"DELETE WHERE {{ <{intent_uri}> ?p ?o . }}") # On nettoie l'intention devenue inutile
                        continue
    
                    # On simule avec un chauffage fort pour être sûr d'y arriver à temps
                    predictions = self.simulation_agent.run_simulation_and_get_results(
                        T_air_initiale=temp_int, 
                        T_mur_initiale=temp_int, # On suppose que les murs sont à la même T° que l'air au départ
                        H_initiale=humi_int, 
                        CO2_initial=co2_int,
                        T_ext=temp_ext, 
                        lumi_ext_lux=lumi_ext,
                        occupants=occupants,
                        puissance_chauffage=1500, # On planifie avec le chauffage fort
                        puissance_apports_internes=apports_internes_W,
                        position_volet=position_volet,
                        consigne_temp=temp_cible
                    )
                    temps_necessaire_min = predictions.get('temps_pour_consigne_min', float('inf'))
    
                    # 3. On calcule l'heure de démarrage optimale
                    if temps_necessaire_min != float('inf'):
                        heure_demarrage_optimale = heure_cible - datetime.timedelta(minutes=temps_necessaire_min)
                        print(f"  [{self.agent_uri.split('#')[-1]}] Calcul : Temps de chauffe estimé: {temps_necessaire_min} min. Démarrage optimal à {heure_demarrage_optimale.strftime('%H:%M')}.")
    
                        if now >= heure_demarrage_optimale:
                            print(f"  [{self.agent_uri.split('#')[-1]}] DÉCISION: C'est l'heure de lancer le préchauffage !")
                            
                            # On TRANSFORME l'intention existante en un ordre concret
                            
                            # a) On change sa description pour refléter l'action immédiate
                            description_action = f"Lancer le préchauffage (planifié) pour atteindre {temp_cible}°C."
                            self.graph.set((intent_uri, EX.description, Literal(description_action)))
    
                            # b) On change son type pour qu'elle devienne une intention de chauffage
                            self.graph.remove((intent_uri, RDF.type, EX.Intention)) # Supprime le type générique
                            self.graph.add((intent_uri, RDF.type, EX.ActiverChauffage))
    
                            # c) On lui attache l'équipement à utiliser
                            self.graph.add((intent_uri, EX.concerneEquipement, EX.RadiateurBureau1))
                            
                            # d) On met à jour son statut au statut final, prêt pour l'Actionneur
                            current_status = self.graph.value(intent_uri, EX.aPourStatut)
                            self.graph.remove((intent_uri, EX.aPourStatut, current_status))
                            self.graph.add((intent_uri, EX.aPourStatut, EX.Statut_Planifiee))
                            
                            print(f"  [{self.agent_uri.split('#')[-1]}] Intention transformée en ordre d'action et marquée 'Planifiée'.")
    
                        else:
                            # Si ce n'est pas l'heure, on ne fait RIEN. L'intention sera ré-évaluée au prochain cycle.
                            print(f"  [{self.agent_uri.split('#')[-1]}] Pas encore l'heure de préchauffer. Attente.")
                            
                    else:
                        print(f"  [{self.agent_uri.split('#')[-1]}] Simulation indique que la consigne ne peut être atteinte. Annulation.")
                        # On supprime l'intention car elle est irréalisable
                        self.graph.update(f"DELETE WHERE {{ <{intent_uri}> ?p ?o . }}")
                    
                    # On passe à l'intention suivante car le cas préchauffage est géré
                    continue
    
            # --- CAS 2 : Intention réactive de chauffage immédiat ---
            elif intent_type == EX.ActiverChauffage:
                print(f"  [{self.agent_uri.split('#')[-1]}] Planification de l'intention de chauffage: {intent_uri.split('#')[-1]}")
                # --- ÉTAPE 1: Collecte complète des données pour la simulation ---
                print(f"  [{self.agent_uri.split('#')[-1]}] Collecte des données d'environnement...")
                temp_int = get_current_sensor_value(self.graph, EX.CapteurTempBureau1, "aPourTemperatureActuelle")
                temp_ext = get_current_sensor_value(self.graph, EX.CapteurTemperatureExterieur, "aPourTemperatureActuelle")
                humi_int = get_current_sensor_value(self.graph, EX.CapteurHumiditeBureau1, "aPourHumiditeActuelle")
                co2_int = get_current_sensor_value(self.graph, EX.CapteurCO2Bureau1, "aPourCO2Actuel")
                lumi_ext = get_current_sensor_value(self.graph, EX.CapteurLuminositeExterieur, "aPourLuminositeActuelle")
                occupants = int(get_current_sensor_value(self.graph, EX.CapteurOccupationBureau1, "aPourOccupationActuelle") or 0)
                    
                # Déterminer la position des volets
                etat_volet = get_equipment_state(self.graph, EX.VoletBureau1)
                position_volet = 1.0 if etat_volet == "Fermee" else 0.0 # 1.0 = 100% fermé
                    
                # --- ÉTAPE 2: Vérification des données et estimation des apports internes ---
                # On ne lance la simulation que si les données essentielles sont là.
                if all(v is not None for v in [temp_int, temp_ext, humi_int, co2_int, lumi_ext]):
                     # Estimation des apports de chaleur internes (personnes, ordinateurs, lampes)
                    apports_internes_W = (occupants * 100) # Apport par personne (chaleur corporelle)
                    apports_internes_W += (occupants * 150) # Apport par équipement (ex: ordinateur)
                        
                    if get_equipment_state(self.graph, EX.LampeBureau1) == "Allume":
                        apports_internes_W += 15 # Apport de la lampe
                        
                    print(f"  [{self.agent_uri.split('#')[-1]}] Données collectées. Apports internes estimés: {apports_internes_W}W.")
    
                    # --- ÉTAPE 3: Lancement des simulations ---
                    # Scénario 1: Chauffage doux
                    pred_doux = self.simulation_agent.run_simulation_and_get_results(
                        T_air_initiale=temp_int, T_mur_initiale=temp_int, H_initiale=humi_int, CO2_initial=co2_int,
                        T_ext=temp_ext, lumi_ext_lux=lumi_ext,
                        occupants=occupants,
                        puissance_chauffage=750,
                        puissance_apports_internes=apports_internes_W,
                        position_volet=position_volet
                        )
                    # Scénario 2: Chauffage fort
                    pred_fort = self.simulation_agent.run_simulation_and_get_results(
                        T_air_initiale=temp_int, T_mur_initiale=temp_int, H_initiale=humi_int, CO2_initial=co2_int,
                        T_ext=temp_ext, lumi_ext_lux=lumi_ext,
                        occupants=occupants,
                        puissance_chauffage=1500,
                        puissance_apports_internes=apports_internes_W,
                        position_volet=position_volet
                        )
                        
                    # --- ÉTAPE 4: Décision via LLM et mise à jour ---
                    system_prompt_plan = "Vous êtes un stratège. Choisissez la meilleure option de chauffage. Répondez par `CHOIX_DOUX` ou `CHOIX_FORT`."
                    human_prompt_plan = f"Dilemme: Chauffer doucement ou fort ?\n- OPTION Doux (750W): Atteint 21°C en {pred_doux['temps_pour_consigne_min']} min. Coût: {pred_doux['energie_consommee_kWh']} kWh.\n- OPTION Fort (1500W): Atteint 21°C en {pred_fort['temps_pour_consigne_min']} min. Coût: {pred_fort['energie_consommee_kWh']} kWh."
                        
                    try:
                        llm_decision = await self.llm.ainvoke({"system_prompt_content": system_prompt_plan, "human_prompt_content": human_prompt_plan})
                        decision = llm_decision.strip().strip('`')
                            
                        plan_description = "Activer le radiateur à 750W (stratégie douce)." if decision == "CHOIX_DOUX" else "Activer le radiateur à 1500W (stratégie forte)."
                        self.graph.set((intent_uri, EX.description, Literal(plan_description, datatype=XSD.string)))
                        print(f"  [{self.agent_uri.split('#')[-1]}] Plan d'action raffiné: '{plan_description}'")
    
                    except Exception as e:
                        print(f"  [{self.agent_uri.split('#')[-1]}] ERREUR Planificateur LLM: {e}. Le plan initial est conservé.")
                
            else:
                print(f"  [{self.agent_uri.split('#')[-1]}] Validation simple pour: {intent_uri.split('#')[-1]}")
    
            # On met à jour le statut de toutes les intentions traitées (sauf le préchauffage qui est géré à part)
            current_status = self.graph.value(intent_uri, EX.aPourStatut)
            if current_status:
                self.graph.remove((intent_uri, EX.aPourStatut, current_status))
                self.graph.add((intent_uri, EX.aPourStatut, EX.Statut_Planifiee))
                print(f"  [{self.agent_uri.split('#')[-1]}] Statut de '{intent_uri.split('#')[-1]}' mis à jour à 'Planifiée'.")


class AgentExecution:
    """
    Rôle : Le "bras armé" du système.
    - Surveille le Tableau Blanc à la recherche d'intentions ayant le statut final "Planifiée".
    - Traduit ces intentions en actions concrètes (dans notre cas, écrit un log d'exécution).
    - Nettoie l'intention du Tableau Blanc une fois l'action terminée pour conclure le cycle.
    """
    
    def __init__(self, graph: Graph, bureau_uri: URIRef, agent_uri: URIRef):
        self.graph = graph
        self.bureau_uri = bureau_uri
        self.agent_uri = agent_uri
        print(f"  [{self.agent_uri.split('#')[-1]}] Agent Actionneur initialisé.")

        
        self.equipment_map = {
             "RadiateurBureau1": EX.RadiateurBureau1,
             "FenetreBureau1": EX.FenetreBureau1,
             "LampeBureau1": EX.LampeBureau1,
             "VentilationBureau1": EX.VentilationBureau1,
             "VoletBureau1": EX.VoletBureau1,
             "ClimatiseurBureau1": EX.ClimatiseurBureau1
        }
    
    def _get_executable_intentions(self):
        """
        [VERSION CHORÉGRAPHIE] Récupère les intentions ayant atteint le statut final ':Statut_Planifiee'.
        C'est le déclencheur de l'agent.
        """
        
        query = f"""
        SELECT ?intention ?description ?equip WHERE {{
            ?intention ex:aPourStatut ex:Statut_Planifiee .
            ?intention ex:viseZone <{self.bureau_uri}> .
            ?intention ex:description ?description .
            OPTIONAL {{ ?intention ex:concerneEquipement ?equip . }}
        }}
        """
        results = self.graph.query(query, initNs=GLOBAL_NAMESPACES)
        intentions = []
        for row in results:
            equipment_name = str(row.equip).split('#')[-1] if row.equip else "None"
            intentions.append({
                "description": str(row.description),
                "equipment": equipment_name,
                "uri": row.intention 
            })
        return intentions

    def executer_actions_et_nettoyer(self, executable_intentions: list):
        """Exécute les actions et supprime les intentions traitées."""
        print(f"  [{self.agent_uri.split('#')[-1]}] Exécution de {len(executable_intentions)} intention(s) planifiée(s)...")
        for intent in executable_intentions:
            action_description = intent["description"]
            equipment_name = intent["equipment"] 
            equipment_uri = self.equipment_map.get(equipment_name)
            intent_uri = intent["uri"]
            
            print(f"  [{self.agent_uri.split('#')[-1]}] EXÉCUTION : '{action_description}' sur '{equipment_name}'.")
            
            # 1. On enregistre l'action dans le log
            add_action_log(self.graph, self.agent_uri, self.bureau_uri, action_description, equipment_uri)
            
            self.graph.update(f"DELETE WHERE {{ <{intent_uri}> ?p ?o . }}")
            print(f"  [{self.agent_uri.split('#')[-1]}] Intention '{intent_uri.split('#')[-1]}' consommée et supprimée.")
            
    async def lire_et_agir(self):
        """Cycle de lecture des intentions planifiées et d'exécution."""
        executable_intentions = self._get_executable_intentions()
        if executable_intentions:
            self.executer_actions_et_nettoyer(executable_intentions)
            
            # On sauvegarde l'état final après action et nettoyage
            try:
                self.graph.serialize(destination=DYNAMIC_WHITEBOARD_FILE, format="turtle", encoding="utf-8")
                print(f"  [{self.agent_uri.split('#')[-1]}] Tableau Blanc sauvegardé après exécution.")
            except Exception as e:
                print(f"  [{self.agent_uri.split('#')[-1]}] ERREUR lors de la sauvegarde du Tableau Blanc : {e}")
        else:
            print(f"  [{self.agent_uri.split('#')[-1]}] Aucune intention planifiée à exécuter.")

## 5. Orchestration et Scénarios de Test

Cette section contient le "chef d'orchestre" (`main` et `run_full_decision_cycle`) qui exécute les agents, ainsi que les fonctions de scénarios de test. Chaque scénario prépare le Tableau Blanc avec une situation spécifique, lance le cycle de décision, et permet d'observer le comportement émergent du système.

In [5]:
# ==============================================================================
# 6. ORCHESTRATION ET SCÉNARIOS DE TEST
# ==============================================================================
# Cette section finale contient le "chef d'orchestre" du programme.
# La fonction `main` initialise tous les agents et les composants,
# puis lance les scénarios de test. Chaque fonction de scénario prépare le
# Tableau Blanc pour une situation spécifique, appelle le cycle de décision
# des agents, et permet d'observer le comportement intelligent du système.


# Dictionnaire global pour stocker les instances de nos agents.
agents = {}

def print_agent_step(agent_name: str, title: str, content: str):
    """
    Fonction utilitaire pour afficher joliment les étapes d'un scénario.
    """
    print("\n" + "="*80)
    print(f"## AGENT/MODULE : {agent_name} | ÉTAPE : {title} ##")
    print("-" * 80)
    # On enlève les balises de style qui ne sont plus interprétées
    content_sans_style = re.sub(r'\[/?\w+\]', '', content)
    print(content_sans_style)
    print("="*80)
    time.sleep(1.5)

def reinitialiser_tableau_blanc():
    """
    Nettoie le Tableau Blanc et le recharge depuis l'ontologie de base.
    Crucial pour s'assurer que chaque test commence dans un environnement propre.
    """
    global g_tableau_blanc
    print("\n" + "---" * 20)
    print("🔄 Réinitialisation du Tableau Blanc Sémantique...")
    g_tableau_blanc = Graph()
    try:
        g_tableau_blanc.parse(ONTOLOGY_FILE_NAME, format="turtle")
        g_tableau_blanc.serialize(destination=DYNAMIC_WHITEBOARD_FILE, format="turtle", encoding="utf-8")
        print("✅ Tableau Blanc réinitialisé avec succès.")
    except Exception as e:
        print(f"❌ ERREUR lors de la réinitialisation du Tableau Blanc : {e}")
    time.sleep(1)

#  Définition des Scénarios de Test 

async def scenario1_simple_conflit_clim_vs_fenetre():
    """
    SCÉNARIO 1 : Teste la GESTION DE CONFLIT.
    - L'AgentConfort veut lancer la clim (trop chaud).
    - L'AgentEcoEnergie s'y oppose car il fait plus frais dehors.
    - L'AgentMediateur doit arbitrer et choisir la solution la plus intelligente.
    """
    reinitialiser_tableau_blanc()
    print("\\n\\n" + "#"*80)
    print("### DÉBUT SCÉNARIO SIMPLE : CONFLIT (CLIMATISEUR VS FENÊTRE) ###")
    print("#"*80)
    
    # PRÉPARATION DU SCÉNARIO 
    description = (
        "La pièce est trop chaude (25°C), créant un besoin de rafraîchissement.\\n"
        "Cependant, il fait plus frais à l'extérieur (19°C).\\n"
        "Le système doit choisir la solution la plus intelligente entre la climatisation et l'ouverture de la fenêtre."
    )
    print_agent_step("Scénario", "État Initial", description)
    
    # L'AgentPerception publie l'état initial du monde
    agents["donnees"].simuler_et_publier_donnees(
        temp_int=25.0, co2=500.0, lumi=1000.0, temp_ext=19.0, 
        humi=55.0, occup=1, lumi_ext=20000.0
    )
    agents["donnees"].simuler_etat_actionneur(EX.RadiateurBureau1, EX.Inactif)
    agents["donnees"].simuler_etat_actionneur(EX.ClimatiseurBureau1, EX.Inactif)
    agents["donnees"].simuler_etat_actionneur(EX.FenetreBureau1, EX.Fermee)
    g_tableau_blanc.serialize(destination=DYNAMIC_WHITEBOARD_FILE, format="turtle")

    # CYCLE DE DÉCISION 
    await agents["confort"].evaluer_et_publier()
    await agents["energie"].evaluer_et_publier(),
    await agents["mediateur"].evaluer_et_arbitrer()
    await agents["planificateur"].evaluer_et_planifier()
    await agents["actionneur"].lire_et_agir()

    print("\\n" + "#"*80)
    print("### FIN SCÉNARIO SIMPLE DE CONFLIT ###")
    print("#"*80)


async def scenario2_synergie():
    """
    SCÉNARIO 2 : Teste la DÉTECTION DE SYNERGIE.
    - L'AgentConfort détecte deux problèmes (trop chaud ET trop de CO2).
    - Il propose deux actions (clim + ventilation).
    - L'AgentStratege doit trouver l'action unique qui résout les deux problèmes (ouvrir la fenêtre).
    """
    reinitialiser_tableau_blanc()
    print("\n\n" + "#"*80)
    print("### DÉBUT SCÉNARIO : DÉTECTION DE SYNERGIE ###")
    print("#"*80)
    
    # PRÉPARATION DU SCÉNARIO ---
    description = (
        "Situation complexe avec deux problèmes simultanés :\n"
        "1. La pièce est trop chaude (24°C).\n"
        "2. Le taux de CO2 est trop élevé (950ppm).\n"
        "Contexte favorable : l'air extérieur est plus frais (19°C)."
    )
    print_agent_step("Scénario", "État Initial", description)
    
    agents["donnees"].simuler_et_publier_donnees(
        temp_int=24.0, co2=950.0, lumi=600.0, temp_ext=19.0, humi=60.0, occup=1, lumi_ext=20000.0
    )
    agents["donnees"].simuler_etat_actionneur(EX.RadiateurBureau1, EX.Inactif)
    agents["donnees"].simuler_etat_actionneur(EX.FenetreBureau1, EX.Fermee)
    agents["donnees"].simuler_etat_actionneur(EX.ClimatiseurBureau1, EX.Inactif)
    agents["donnees"].simuler_etat_actionneur(EX.VentilationBureau1, EX.Inactif)
    agents["donnees"].simuler_etat_actionneur(EX.VoletBureau1, EX.Ouvert) 

    g_tableau_blanc.serialize(destination=DYNAMIC_WHITEBOARD_FILE, format="turtle")

    # CYCLE DE DÉCISION ---
    print("\n--- Cycle de Décision ---")
    await agents["confort"].evaluer_et_publier()
    await agents["stratege"].evaluer_et_optimiser()
    await agents["planificateur"].evaluer_et_planifier()
    await agents["actionneur"].lire_et_agir()

    print("\n" + "#"*80)
    print("### FIN SCÉNARIO SYNERGIE ###")
    print("#"*80)


async def scenario3_simple_proactif():
    """
    SCÉNARIO 3 : Teste la GESTION PROACTIVE.
    - Un événement futur est créé (arrivée d'un occupant).
    - L'AgentCalendrier doit le détecter et créer une intention de préchauffage.
    - L'AgentPlanificateur doit calculer un plan mais décider d'attendre le bon moment.
    """
    reinitialiser_tableau_blanc()
    print("\\n\\n" + "#"*80)
    print("### DÉBUT SCÉNARIO SIMPLE : GESTION PROACTIVE (PRÉCHAUFFAGE) ###")
    print("#"*80)
    
    # PRÉPARATION DU SCÉNARIO 
    
    # On simule une arrivée dans 20 minutes pour que ce soit dans la fenêtre de détection.
    now = datetime.datetime.now(datetime.timezone.utc)
    meeting_time = now + datetime.timedelta(minutes=20)
    
    # On CRÉE dynamiquement l'événement pour le test
    event_uri = EX["Arrivee_Test_Dynamique"]
    g_tableau_blanc.add((event_uri, RDF.type, EX.EvenementCalendrier))
    g_tableau_blanc.add((event_uri, EX.heureDebut, Literal(meeting_time, datatype=XSD.dateTime)))
    g_tableau_blanc.add((event_uri, EX.viseZone, EX.Bureau1))
    g_tableau_blanc.add((event_uri, EX.concerneOccupant, EX.OccupantJean))
    

    description = (
        f"La pièce est froide (17°C) et inoccupée.\\n"
        f"Un événement est planifié pour une arrivée à {meeting_time.strftime('%H:%M')}.\\n"
        "Le système doit calculer le bon moment pour démarrer le chauffage et l'exécuter."
    )
    print_agent_step("Scénario", "État Initial", description)
    
    agents["donnees"].simuler_et_publier_donnees(
        temp_int=17.0, co2=450.0, lumi=300.0, temp_ext=10.0, 
        humi=55.0, occup=0, lumi_ext=5000.0
    )
    agents["donnees"].simuler_etat_actionneur(EX.RadiateurBureau1, EX.Inactif)
    g_tableau_blanc.serialize(destination=DYNAMIC_WHITEBOARD_FILE, format="turtle")



    # --- CYCLE DE DÉCISION ---
    await agents["calendrier"].evaluer_et_creer_intentions_proactives()
    await agents["confort"].evaluer_et_publier()
    await agents["mediateur"].evaluer_et_arbitrer()
    await agents["stratege"].evaluer_et_optimiser()
    await agents["planificateur"].evaluer_et_planifier()
    await agents["actionneur"].lire_et_agir()

    print("\\n" + "#"*80)
    print("### FIN SCÉNARIO SIMPLE PROACTIF ###")
    print("#"*80)


async def main():
    """
    Fonction principale qui initialise tous les composants du système
    et lance la séquence de tests.
    """
    global agents
    print("\n--- Initialisation du Système Multi-Agents ---")
    load_dotenv()
    # Configuration et initialisation des services (LLM, Simulation)
    OPENROUTER_API_KEY_VALUE = os.getenv("OPENROUTER_API_KEY")
    GROQ_API_KEY_VALUE = os.getenv("GROQ_API_KEY")
    openrouter_config = {"api_key": OPENROUTER_API_KEY_VALUE, "base_url": "https://openrouter.ai/api/v1", "model_name": "mistralai/mistral-7b-instruct:free"}
    groq_config = {"api_key": GROQ_API_KEY_VALUE, "base_url": "https://api.groq.com/openai/v1", "model_name": "llama-3.1-8b-instant"}
    llm_manager = LLMManager(openrouter_config, groq_config)
    volume_piece = get_physical_property(g_tableau_blanc, EX.Bureau1, EX.volume) or 30.0
    capacite_thermique_air = get_physical_property(g_tableau_blanc, EX.Bureau1, EX.capaciteThermique) or 100000.0
    resistance_murs = get_physical_property(g_tableau_blanc, EX.Bureau1, EX.resistanceThermiqueMurs) or 0.1
    capacite_thermique_murs = get_physical_property(g_tableau_blanc, EX.Bureau1, EX.capaciteThermiqueMurs) or 5000000.0
    surface_vitree = get_physical_property(g_tableau_blanc, EX.Bureau1, EX.surfaceVitree) or 3.5
    facteur_solaire = get_physical_property(g_tableau_blanc, EX.Bureau1, EX.facteurSolaire) or 0.6
    agent_simulation = AgentSimulation(volume_piece=volume_piece, capacite_thermique_air=capacite_thermique_air, capacite_thermique_murs=capacite_thermique_murs, resistance_thermique_murs=resistance_murs, surface_vitree_m2=surface_vitree, facteur_solaire=facteur_solaire)

    # Création de chaque agent et stockage dans le dictionnaire global
    agents = {
        "donnees": AgentPerception(g_tableau_blanc, EX.Bureau1, EX.AgentPerception1, EX.CapteurTempBureau1, EX.CapteurCO2Bureau1, EX.CapteurLuminositeBureau1, EX.CapteurHumiditeBureau1, EX.CapteurOccupationBureau1, EX.CapteurTemperatureExterieur, EX.CapteurLuminositeExterieur, EX.RadiateurBureau1, EX.FenetreBureau1, EX.LampeBureau1, EX.VentilationBureau1, EX.VoletBureau1),
        "confort": AgentConforts(g_tableau_blanc, EX.Bureau1, EX.AgentConfort1, EX.OccupantJean, llm_manager),
        "energie": AgentEcoEnergie(g_tableau_blanc, EX.Bureau1, EX.AgentEcoEnergie1, llm_manager),
        "mediateur": AgentMediateur(g_tableau_blanc, EX.AgentMediateur1, llm_manager),
        "stratege": AgentStratege(g_tableau_blanc, EX.AgentStratege1, llm_manager),
        "planificateur": AgentPlanificateur(g_tableau_blanc, EX.AgentPlanificateur1, llm_manager, agent_simulation),
        "actionneur": AgentExecution(g_tableau_blanc, EX.Bureau1, EX.AgentExecution1),
        "calendrier": AgentCalendrier(g_tableau_blanc, EX.AgentCalendrier1)
    }
    print("✅ Tous les agents ont été initialisés.")


    await scenario1_simple_conflit_clim_vs_fenetre()
    print("\n--- PAUSE (5s) ---")
    time.sleep(5)
    
    await scenario2_synergie()
    print("\n--- PAUSE (5s) ---")
    time.sleep(5)
    
    await scenario3_simple_proactif()
    
   

if __name__ == "__main__":
    # Ce bloc gère le lancement de la boucle événementielle asynchrone,
    # nécessaire pour faire fonctionner les agents. Il est compatible
    # avec un script Python classique et un environnement Jupyter Notebook.
    try:
        loop = asyncio.get_running_loop()
        if loop.is_running():
            print("Boucle asyncio détectée. Lancement des scénarios comme une tâche.")
            loop.create_task(main())
        else:
            asyncio.run(main())
    except RuntimeError:
        print("Aucune boucle asyncio. Lancement des scénarios avec asyncio.run().")
        asyncio.run(main())


Boucle asyncio détectée. Lancement des scénarios comme une tâche.

--- Initialisation du Système Multi-Agents ---
  [Système] LLMManager initialisé avec OpenRouter, Groq et parsing JSON.
  [Système] AgentSimulation (modèle avancé) initialisé.
  [AgentPerception1] Agent Bâtiments (Simulateur Complet) initialisé.
  [AgentConfort1] Agent Conforts (Logique Fiable) initialisé.
  [AgentEcoEnergie1] Agent EcoEnergie initialisé pour Bureau1.
  [AgentMediateur1] Agent Médiateur initialisé.
  [AgentStratege1] Agent Stratège (LLM-driven) initialisé.
  [AgentPlanificateur1] Agent Planificateur initialisé.
  [AgentExecution1] Agent Actionneur initialisé.
  [AgentCalendrier1] Agent Calendrier initialisé.
✅ Tous les agents ont été initialisés.

------------------------------------------------------------
🔄 Réinitialisation du Tableau Blanc Sémantique...
✅ Tableau Blanc réinitialisé avec succès.
\n\n################################################################################
### DÉBUT SCÉNARIO SIM